In [1]:
# _*_ coding: utf-8 _*_
# @Time: 2021/12/09 16:05 
# @Author: yuyongsheng
# @Software: PyCharm
# @Description: 南方医院丙戊酸：1. 数据处理

# 原始数据预处理

## 导入程序包

In [2]:
# 导入程序包
import pymysql as MySQLDB
import pandas as pd
import numpy as np

import re
import sys
import os
project_path = os.getcwd()
print(project_path)

D:\PycharmProjects\VPA_southern_hospital


## 导入预定义函数

In [3]:
# 字符串转换为时间格式
import datetime
def str_to_datetime(x):
    try:
        a = datetime.datetime.strptime(x, "%d/%m/%Y %H:%M:%S")
        return a
    except:
        return np.NaN

In [4]:
# 过滤文字
import numpy as np
def filter_string(df,feature):
    # 过滤文字!!!!!!!!!!!!!!!!!!!!!!!!!!!
    df=df[df[feature].str.contains('\d')]
    return df

In [5]:
# 过滤特殊字符
def filter_spec_character(df,feature):
    df[feature]=df[feature].apply(lambda x: re.sub('<|>|\+|\/|-','',str(x)))
    return df

In [6]:
# 过滤异常大值
def filter_huge_value(df,feature,median_value=1):
    # 过滤异常大值!!!!!!!!!!!!!!!!!!!!!!!!!!
    if not median_value:
        median_value=df[feature].median()
    df[feature]=df[feature].apply(lambda x: np.nan if x<0 else x if float(x) <= 100 * median_value else 100*median_value)
    df=df[df[feature].notnull()]
    return df

In [7]:
# 使用随机森林对缺失值进行插补
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
def missing_value_interpolation(df,missing_list=[]):
    df = df.reset_index(drop=True)
    # 提取存在缺失值的列名
    if not missing_list:
        for i in df.columns:
            if df[i].isnull().sum() > 0:
                missing_list.append(i)
    missing_list_copy = missing_list.copy()
    # 用该列未缺失的值训练随机森林，然后用训练好的rf预测缺失值
    for i in range(len(missing_list)):
        name=missing_list[0]
        df_missing = df[missing_list_copy]
        # 将其他列的缺失值用0表示。
        missing_list.remove(name)
        for j in missing_list:
            df_missing[j]=df_missing[j].astype('str').apply(lambda x: 0 if x=='nan' else x)
        df_missing_is = df_missing[df_missing[name].isnull()]
        df_missing_not = df_missing[df_missing[name].notnull()]
        y = df_missing_not[name]
        x = df_missing_not.drop([name],axis=1)
        # 列出参数列表
        tree_grid_parameter = {'n_estimators': list((10, 50, 100, 150, 200))}
        # 进行参数的搜索组合
        grid = GridSearchCV(RandomForestRegressor(),param_grid=tree_grid_parameter,cv=3)
        #rfr=RandomForestRegressor(random_state=0,n_estimators=100,n_jobs=-1)
        #根据已有数据去拟合随机森林模型
        grid.fit(x, y)
        rfr = RandomForestRegressor(n_estimators=grid.best_params_['n_estimators'])
        rfr.fit(x, y)
        #预测缺失值
        predict = rfr.predict(df_missing_is.drop([name],axis=1))
        #填补缺失值
        df.loc[df[name].isnull(),name] = predict
    return df

## 连接数据库

In [8]:
# 丙戊酸-双相障碍1
# 连接数据库
conn = MySQLDB.connect(host='192.168.0.37', port=3306, user='root', password='111111', db='ndyy_vpa_sxza_1', charset='UTF8')
# conn = MySQLDB.connect(host='localhost', port=3306, user='root', password='123456', db='上海六院', charset='UTF8')
cursor = conn.cursor()
cursor.execute("Select version()")
for i in cursor:
    print(i)

('8.0.22',)


## 读取doctor_order数据库表

In [9]:
# 丙戊酸-双相障碍v1.0和丙戊酸-双相障碍v2.0
# 从df_doctor_order数据库读入数据
# 丙戊酸long_d_order不准，不要
try:
    sql = 'select patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc \
    from doctor_order \
    UNION \
    select patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc \
    from ndyy_vpa_sxza_2.doctor_order;'
    df_doctor_order = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [10]:
print(df_doctor_order.shape)
print(df_doctor_order['patient_id'].nunique())

(26568, 10)
218


In [11]:
# 保存doctor_order用药记录
df_doctor_order.to_excel(project_path+'/data/pre_processed_raw_data/df_doctor_order.xlsx')

In [12]:
# 手工处理临时医嘱和长期医嘱，根据药物属性合并删除重复数据，一天只有一条临时医嘱和长期医嘱
# # 设置patient_id，case_no字段数据数据格式，不然无法匹配id
df_doctor_order=pd.read_excel(project_path+'/data/pre_processed_raw_data/df_doctor_order_手工处理临嘱和长嘱.xlsx',
                              converters={'patient_id':str,'case_no':str})
if 'Unnamed: 0' in df_doctor_order.columns:
    df_doctor_order = df_doctor_order.drop(['Unnamed: 0'], axis=1)

## 读取inp_record住院记录

In [13]:
# 从inp_record表读入数据
try:
    sql = 'select patient_id,case_no,adm_date,care_area,dis_date,age from inp_record \
     UNION \
    select patient_id,case_no,adm_date,care_area,dis_date,age from ndyy_vpa_sxza_2.inp_record;'
    df_inp_record = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [14]:
print(df_inp_record.shape)
print(df_inp_record['patient_id'].nunique())

(241, 6)
218


In [15]:
#用来判断某列是否有缺失值
df_inp_record.isnull().any() 

patient_id    False
case_no       False
adm_date      False
care_area     False
dis_date       True
age           False
dtype: bool

In [16]:
df_inp_record[df_inp_record['dis_date'].isnull()]

,patient_id,case_no,adm_date,care_area,dis_date,age
0,6873870,10001302,2020-08-15 17:12:30,JSKXLHLDY-心理二区护理单元江高,NaT,28
2,6949939,10019847,2020-08-17 20:28:47,JSKXLHLDY-心理二区护理单元江高,NaT,27
4,369329,10028911,2020-08-18 15:12:28,JSKXLHLDY-心理二区护理单元江高,NaT,33
27,8043274,10914758,2020-11-18 17:38:49,JSKXLHLDY-心理二区护理单元江高,NaT,29


In [17]:
# 查看重复数据
df_inp_record[df_inp_record.duplicated(subset=['patient_id','adm_date'],keep='first')]

,patient_id,case_no,adm_date,care_area,dis_date,age


In [18]:
df_inp_record.to_excel(project_path+'/data/pre_processed_raw_data/df_inp_record.xlsx')

## 读取diagnostic_record

In [19]:
# 从medical_record表读入数据
try:
    sql = 'select patient_id,case_no,record_date,diagnostic_type,diagnostic_content from diagnostic_record \
        UNION \
        select patient_id,case_no,record_date,diagnostic_type,diagnostic_content from ndyy_vpa_sxza_2.diagnostic_record;'
    df_diagnostic_record = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [20]:
print(df_diagnostic_record.shape)
print(df_diagnostic_record['patient_id'].nunique())

(1873, 5)
359


In [21]:
#用来判断某列是否有缺失值
df_diagnostic_record.isnull().any() 

patient_id            False
case_no                True
record_date           False
diagnostic_type       False
diagnostic_content    False
dtype: bool

In [22]:
# 查看重复数据
df_diagnostic_record[df_diagnostic_record.duplicated(subset=['patient_id','case_no','record_date','diagnostic_content'],keep='first')]

,patient_id,case_no,record_date,diagnostic_type,diagnostic_content


In [23]:
writer=pd.ExcelWriter(project_path+'/data/pre_processed_raw_data/df_diagnostic_record.xlsx')
df_diagnostic_record.to_excel(writer)
writer.save()

## 读取test_result数据表

### test_record

In [24]:
# 从数据库读入test_record数据
try:
    sql = 'select test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time from test_record \
    UNION \
    select test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time from ndyy_vpa_sxza_2.test_record;'
    df_test_record = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [25]:
print(df_test_record.shape)
print(df_test_record['patient_id'].nunique())

(9953, 7)
364


In [26]:
df_test_record

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time
0,800001202207||B001||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35
1,800001202207||B002||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35
2,800001202207||B005||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35
3,800001202207||B006||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35
4,800001202207||B007||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35
...,...,...,...,...,...,...,...
9948,800014409316||B008||1,8004050,None,2021-12-24 11:55:00,1:双相情感障碍 2:锥体外系综合征,血清,2021-12-24 09:20:18
9949,800014409316||B010||1,8004050,None,2021-12-24 11:55:00,1:双相情感障碍 2:锥体外系综合征,血清,2021-12-24 09:20:47
9950,800014409316||B178||1,8004050,None,2021-12-24 11:55:00,1:双相情感障碍 2:锥体外系综合征,血清,2021-12-24 09:20:47
9951,800014409317||A083||1,8004050,None,2021-12-24 09:42:00,1:双相情感障碍 2:锥体外系综合征,静脉血,2021-12-24 09:20:18


In [27]:
#用来判断某列是否有缺失值
df_test_record.isnull().any() 

test_record_id        False
patient_id            False
case_no                True
test_date             False
clinical_diagnosis     True
sample_type           False
sample_time            True
dtype: bool

In [28]:
# 保存test_record
df_test_record.to_excel(project_path+'/data/pre_processed_raw_data/df_test_record.xlsx')

### test_result

In [29]:
# 从数据库读入test_result数据
try:
    sql = 'select test_record_id,project_name,test_result,result_unit,is_normal,refer_scope,synonym from test_result\
    UNION \
    select test_record_id,project_name,test_result,result_unit,is_normal,refer_scope,synonym from ndyy_vpa_sxza_2.test_result;'
    df_test_result = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [30]:
print(df_test_result.shape)

(61713, 7)


In [31]:
#用来判断某列是否有缺失值
df_test_result.isnull().any() 

test_record_id    False
project_name      False
test_result       False
result_unit        True
is_normal          True
refer_scope        True
synonym            True
dtype: bool

In [32]:
# 查看重复数据
df_test_result[df_test_result.duplicated(subset=['test_record_id','project_name','test_result'],keep='first')]

,test_record_id,project_name,test_result,result_unit,is_normal,refer_scope,synonym
59467,800013465956||B042||1,免疫球蛋白,阴性(-),None,N,阴性(-),IgA_gd
59468,800013465956||B042||1,免疫球蛋白,阴性(-),None,N,阴性(-),IgM_gd


In [33]:
# 删除test_result数据
df_test_result=df_test_result.drop_duplicates(subset=['test_record_id','project_name','test_result'],keep='first')
df_test_result=df_test_result.reset_index(drop=True)

In [34]:
print(df_test_result.shape)

(61711, 7)


In [35]:
# 删除<>号
df_test_result=filter_spec_character(df_test_result,'test_result')

In [36]:
print(df_test_result.shape)

(61711, 7)


In [37]:
# 保存test_result
df_test_result.to_excel(project_path+'/data/pre_processed_raw_data/df_test_result.xlsx')

### test_record+test_result

In [38]:
df_test=pd.merge(df_test_record,df_test_result,on=['test_record_id'],how='inner')

In [39]:
df_test.head()

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym
0,800001202207||B001||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35,总二氧化碳,28.7,mmol/L,N,21.0-31.0,TCO2
1,800001202207||B001||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35,尿酸,364,μmol/L,N,208-428,UA
2,800001202207||B001||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35,肌酐,86,μmol/L,N,53-123,CR
3,800001202207||B001||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35,尿素,4.7,mmol/L,N,2.8-7.2,UREA
4,800001202207||B002||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35,天门冬氨酸氨基转移酶,25,U/L,N,15-40,AST


In [40]:
print(df_test.shape)
print(df_test['patient_id'].nunique())

(61711, 13)
364


In [41]:
# 保存检测数据
df_test.to_excel(project_path+'/data/pre_processed_raw_data/df_test.xlsx')

## 体检数据

In [42]:
# 从数据库读入体检exam数据
try:
    sql = 'select patientId,examDateTime,normalInd,description from exam_record \
        UNION select patientId,examDateTime,normalInd,description from ndyy_vpa_sxza_2.exam_record;'
    df_exam = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [43]:
# 删除空数据
df_exam=df_exam[df_exam['description'].notnull()].reset_index(drop=True)

In [44]:
# 重命名
df_exam=df_exam.rename(columns={'patientId':'patient_id',
                       'examDateTime':'exam_time'})

In [45]:
print(df_exam.shape)
print(df_exam['patient_id'].nunique())

(429, 4)
192


In [46]:
df_exam

,patient_id,exam_time,normalInd,description
0,8242965,2020-09-01 00:33:40,阴性,两侧胸廓对称、所见骨质未见异常；两侧肺野透过度正常，未见异常密度增高影；两肺纹理略增粗、增多...
1,8242965,2020-09-01 00:33:40,阳性,双肾大小形态正常，轮廓清晰，包膜完整，实质回声均匀，集合系统回声正常，双侧输尿管不扩张。膀胱...
2,7006940,2020-09-01 00:33:40,阳性,主动脉扩张、迂曲、延长，主动脉结向左突出，见弧状钙化影；两侧肺野透光度正常，未见明确实变影；...
3,7006940,2020-09-01 00:33:40,阳性,肝脏形态大小正常，包膜完整光滑，肝内实质回声细密稍增强，肝内血管及胆管显示欠清晰，门静脉、胆...
4,8413999,2020-09-03 00:33:40,阴性,两侧胸廓对称、所见骨质未见异常；两侧肺野透过度正常，未见异常密度增高影；两肺纹理略增粗、增多...
...,...,...,...,...
424,9080293,2021-10-22 10:32:30,阳性,头部平扫示双侧大脑半球对称，灰白质对比正常，脑实质内未见局灶性异常密度影。各脑室、脑...
425,7692024,2021-12-07 15:07:35,阴性,两侧胸廓对称、所见骨质未见异常；两侧肺野透过度正常，未见异常密度增高影；两肺纹理略增粗、增多...
426,7692024,2021-12-09 08:38:12,阴性,肝脏形态大小正常，包膜完整光滑，下缘角锐利，肝内实质回声均匀，肝内血管及胆管显示清晰，走行正...
427,8505473,2021-12-13 15:10:42,阴性,两侧胸廓对称、所见骨质未见异常；两侧肺野透过度正常，未见异常密度增高影；两肺纹理清晰，无增粗...


In [47]:
df_exam.to_excel(project_path+'/data/pre_processed_raw_data/df_exam.xlsx')

## 提取人口学特征

### gender、race

In [48]:
# 从patient_info提取gender,race,birth_year
try:
    sql = 'select patient_id,gender,race,birth_year from patient_info \
        UNION select patient_id,gender,race,birth_year from ndyy_vpa_sxza_2.patient_info;'
    df_patient_info = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [49]:
print(df_patient_info.shape)
print(df_patient_info['patient_id'].nunique())

(364, 4)
364


In [50]:
df_patient_info['gender']=df_patient_info['gender'].apply(lambda x: 1 if re.match('男',x) else 0)

In [51]:
#用来判断某列是否有缺失值
df_patient_info.isnull().any() 

patient_id    False
gender        False
race           True
birth_year    False
dtype: bool

In [52]:
# 查看重复数据
df_patient_info[df_patient_info.duplicated(subset=['patient_id','gender','race'],keep='first')]

,patient_id,gender,race,birth_year


In [53]:
print(df_patient_info.shape)
print(df_patient_info['patient_id'].nunique())

(364, 4)
364


In [54]:
df_patient_info.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_patient_info.xlsx')

### age

In [55]:
df_age=df_inp_record[['patient_id','case_no','age']]

In [56]:
print(df_age.shape)
print(df_age['patient_id'].nunique())

(241, 3)
218


In [57]:
df_age

,patient_id,case_no,age
0,6873870,10001302,28
1,547307,10005574,25
2,6949939,10019847,27
3,7765586,10024628,16
4,369329,10028911,33
...,...,...,...
236,9080293,14400275,24
237,9424074,14701280,27
238,9442416,14712892,25
239,7692024,14900947,21


In [58]:
df_age.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_age.xlsx')

### height

In [59]:
# 从patient_sign_record提取height、weight
try:
    sql = 'select patient_id,case_no,record_date,sign_type,record_content,record_unit from patient_sign_record \
        UNION select patient_id,case_no,record_date,sign_type,record_content,record_unit from ndyy_vpa_sxza_2.patient_sign_record;'
    df_patient_sign_record = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [60]:
df_patient_sign_record.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_patient_sign_record.xlsx')

In [61]:
# 提取身高信息
df_height=df_patient_sign_record[df_patient_sign_record['sign_type'].str.contains('身高')]

In [62]:
print(df_height.shape)
print(df_height['patient_id'].nunique())

(252, 6)
217


In [63]:
df_height

,patient_id,case_no,record_date,sign_type,record_content,record_unit
0,6873870,10001302,2020-08-15 14:00:00,身高(cm),153,cm
79,547307,10005574,2020-08-16 14:00:00,身高(cm),162,cm
143,6949939,10019847,2020-08-17 14:00:00,身高(cm),160,cm
213,7765586,10024628,2020-08-18 14:00:00,身高(cm),168,cm
262,369329,10028911,2020-08-18 14:00:00,身高(cm),159,cm
...,...,...,...,...,...,...
19155,9080293,14400275,2021-10-20 14:00:00,身高(cm),175,cm
19249,9424074,14701280,2021-11-15 14:00:00,身高(cm),168,cm
19313,9442416,14712892,2021-11-16 14:00:00,身高(cm),171,cm
19383,7692024,14900947,2021-12-02 14:00:00,身高(cm),154,cm


In [64]:
# 过滤文字
df_height=filter_string(df_height,'record_content')

In [65]:
print(df_height.shape)
print(df_height['patient_id'].nunique())

(248, 6)
217


In [66]:
# 查看重复数据
df_height[df_height.duplicated(subset=['patient_id','case_no','record_content'])]

,patient_id,case_no,record_date,sign_type,record_content,record_unit
6324,8881895,14103871,2021-10-13 14:00:00,身高(cm),168,cm
6425,8881895,14103871,2021-10-27 14:00:00,身高(cm),168,cm
6473,8881895,14103871,2021-11-03 14:00:00,身高(cm),168,cm
7508,7475493,4191397,2018-08-13 06:00:00,身高(cm),173,cm
7545,7475493,4191397,2018-08-20 14:00:00,身高(cm),173,cm
7786,6679951,4267891,2018-08-22 18:00:00,身高(cm),156,cm
7790,6679951,4267891,2018-08-22 14:00:00,身高(cm),156,cm
18043,8275190,9391429,2020-06-23 14:00:00,身高(cm),155,cm


In [67]:
# 排序
df_height=df_height.sort_values(by=['patient_id','case_no','record_date'],ascending=[True,True,True])
# 删除重复数据
df_height=df_height.drop_duplicates(subset=['patient_id','case_no','record_content'],keep='last')

In [68]:
print(df_height.shape)
print(df_height['patient_id'].nunique())

(240, 6)
217


In [69]:
df_height.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_height.xlsx')

### weight

In [70]:
# 提取体重信息
df_weight=df_patient_sign_record[df_patient_sign_record['sign_type'].str.contains('体重')]

In [71]:
print(df_weight.shape)
print(df_weight['patient_id'].nunique())

(820, 6)
218


In [72]:
df_weight

,patient_id,case_no,record_date,sign_type,record_content,record_unit
11,6873870,10001302,2020-08-15 14:00:00,体重(kg),52,kg
30,6873870,10001302,2020-08-22 06:00:00,体重(kg),52,kg
60,6873870,10001302,2020-08-29 14:00:00,体重(kg),52,kg
90,547307,10005574,2020-08-16 14:00:00,体重(kg),52,kg
111,547307,10005574,2020-08-23 06:00:00,体重(kg),52,kg
...,...,...,...,...,...,...
19370,9442416,14712892,2021-11-30 06:00:00,体重(kg),71,kg
19394,7692024,14900947,2021-12-02 14:00:00,体重(kg),85,kg
19413,7692024,14900947,2021-12-09 14:00:00,体重(kg),54,kg
19449,8505473,15026052,2021-12-11 14:00:00,体重(kg),46,kg


In [73]:
# 过滤文字
df_weight=filter_string(df_weight,'record_content')

In [74]:
print(df_weight.shape)
print(df_weight['patient_id'].nunique())

(815, 6)
218


In [75]:
# 过滤异常值
df_weight=df_weight[~ df_weight['record_content'].str.contains('0')]

In [76]:
print(df_weight.shape)
print(df_weight['patient_id'].nunique())

(688, 6)
190


In [77]:
# 查看重复数据
df_weight[df_weight.duplicated(subset=['patient_id','case_no','record_content'])]

,patient_id,case_no,record_date,sign_type,record_content,record_unit
30,6873870,10001302,2020-08-22 06:00:00,体重(kg),52,kg
60,6873870,10001302,2020-08-29 14:00:00,体重(kg),52,kg
111,547307,10005574,2020-08-23 06:00:00,体重(kg),52,kg
140,547307,10005574,2020-08-30 14:00:00,体重(kg),52,kg
177,6949939,10019847,2020-08-24 14:00:00,体重(kg),45,kg
...,...,...,...,...,...,...
19215,9080293,14400275,2021-11-03 14:00:00,体重(kg),57,kg
19238,9080293,14400275,2021-11-10 06:00:00,体重(kg),57,kg
19343,9442416,14712892,2021-11-23 14:00:00,体重(kg),71,kg
19370,9442416,14712892,2021-11-30 06:00:00,体重(kg),71,kg


In [78]:
# 排序
df_weight=df_weight.sort_values(by=['patient_id','case_no','record_date'],ascending=[True,True,True])
# 删除重复数据
df_weight=df_weight.drop_duplicates(subset=['patient_id','case_no','record_content'],keep='last')

In [79]:
print(df_weight.shape)
print(df_weight['patient_id'].nunique())

(246, 6)
190


In [80]:
df_weight.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_weight.xlsx')

# 纳排

## 纳入标准

### 提取丙戊酸用药

In [81]:
# 提取甲氨蝶呤的用药数据
# 注意药物缩写和药物音译名，wtf
df_VPA = df_doctor_order[df_doctor_order['drug_name'].str.contains('丙戊酸') & (~df_doctor_order['drug_name'].str.contains('药材科'))]

In [82]:
# 排序
df_VPA=df_VPA.sort_values(['patient_id','case_no','start_datetime'])

In [83]:
print(df_VPA.shape)
print(df_VPA['patient_id'].nunique())

(776, 10)
218


In [84]:
# 保存丙戊酸用药数据
df_VPA.to_excel(project_path+'/data/processed_data/df_2.1.1_VPA.xlsx')

### 双相情感障碍者

In [85]:
# 从diagnostic_record中提取双相情感障碍
df_BD=df_diagnostic_record[df_diagnostic_record['diagnostic_content'].str.contains('双相')]

In [86]:
df_BD['diagnostic_content'].unique()

array(['双相情感障碍，目前为伴有精神病性症状的躁狂发作', '双相情感障碍，目前为轻度或中度抑郁发作',
       '双相情感障碍目前为不伴精神病性症状躁狂发作', '双相情感障碍，目前为轻躁狂发作', '双相情感障碍，目前为中度抑郁发作',
       '双相情感障碍，目前为缓解状态', '双相情感障碍', '双相情感障碍，目前为混合性发作',
       '双相情感障碍目前为伴精神病的重度抑郁发作', '双相情感障碍目前为不伴精神病的重度抑郁发作', '非典型双相情感障碍',
       '双相情感障碍，目前为不伴有精神病性症状的重度抑郁发作', '双相情感障碍,快速循环型',
       '双相情感障碍，目前为躁狂发作，伴有精神病性症状', '双相情感障碍，目前为不伴有精神病性症状的躁狂发作', '双相性情感障碍',
       '双相情感障碍（恢复期）', '双相情感障碍,目前为中度抑郁发作', '双相情感障碍，伴有精神病性症状，目前为躁狂发作',
       '双相情感障碍不伴有精神病性症状的躁狂发作', '双相性情感障碍?', '双相情感障碍目前为伴精神病性症状的躁狂发作？',
       '双相情感障碍，目前为混合性发作？', '双相情感障碍，目前为重度抑郁发作', '双相情感障碍？',
       '双相情感障碍，目前为伴有精神病性症状的重度抑郁发作', '非典型双相情感障碍?',
       '双相情感障碍，目前为重度抑郁发作，伴有精神病性症'], dtype=object)

In [87]:
print(df_BD.shape)
print(df_BD['patient_id'].nunique())

(879, 5)
354


In [88]:
# 纳入双相情感障碍
df_VPA_BD=df_VPA[df_VPA['patient_id'].isin(df_BD['patient_id'].unique())]

In [89]:
print(df_VPA_BD.shape)
print(df_VPA_BD['patient_id'].nunique())

(767, 10)
214


In [90]:
df_VPA_BD_not=df_VPA[~ df_VPA['patient_id'].isin(df_BD['patient_id'].unique())]
df_VPA_BD_not['patient_id'].unique()

array(['7506303', '7512212', '7528046', '7680010'], dtype=object)

In [91]:
# ### 年龄筛选：18-65岁

# df_age_adult=df_age[(df_age['age']>=18) & (df_age['age']<=65)]

# df_age_adult

# print(df_age_adult.shape)
# print(df_age_adult['patient_id'].nunique())

# # 纳入年龄限制
# df_VPA_age=df_VPA_BD[df_VPA_BD['patient_id'].isin(df_age_adult['patient_id'].unique())]

# print(df_VPA_age.shape)
# print(df_VPA_age['patient_id'].nunique())

# df_VPA_age.to_excel(project_path+'/data/processed_data/df_temp_VPA_age.xlsx')

### 达到血药浓度稳态

#### 计算丙戊酸日剂量

In [92]:
df_VPA_dosage=df_VPA_BD.copy()
# 清洗单次日剂量dosage，100ml--5g，1ml--0.05g
df_VPA_dosage['dosage']=df_VPA_dosage['dosage'].apply(lambda x: float(x.replace('ml', ''))*0.05 if 'ml' in x else x.replace('g', ''))

In [93]:
df_VPA_dosage.frequency.unique()

array(['1/日', '2/日', '1/早', 'ONCE', 'Tid', '1/晚', '1/12小时', '3/日（餐前）',
       '1/午', '1/日(餐前)'], dtype=object)

In [94]:
# 清洗频次
one=['1/午','ONCE','1/日','1/日(餐前)','1/早','1/晚']
two=['1/12小时','2/日']
three=['Tid','3/日（餐前）']
df_VPA_dosage['frequency']=df_VPA_dosage['frequency'].apply(lambda x: 1 if x in one else
                                                                2 if x in two else
                                                                3 if x in three else x)

In [95]:
# 计算日剂量
df_VPA_dosage['日剂量']=df_VPA_dosage['dosage'].astype('float') * df_VPA_dosage['frequency'].astype('float')

In [96]:
df_VPA_dosage

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量
5276,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2021-03-29 19:14:29,NaT,核实,0.50
5277,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-03-31 09:09:50,NaT,核实,1.00
5289,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-07 09:10:18,NaT,核实,0.75
5352,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-09 19:49:24,NaT,核实,0.75
8053,1173560,13397293,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-07-21 16:21:42,NaT,核实,1.00
...,...,...,...,...,...,...,...,...,...,...,...
10182,9307105,14184132,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-09-30 10:29:44,NaT,核实,1.00
10628,9424074,14701280,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-11-15 18:44:04,NaT,核实,1.00
10777,9442416,14712892,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2021-11-17 10:47:51,NaT,核实,0.50
10889,9442416,14712892,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-11-19 10:31:22,NaT,核实,1.00


In [97]:
print(df_VPA_dosage.shape)
print(df_VPA_dosage['patient_id'].nunique())
print(df_VPA_dosage['case_no'].nunique())

(767, 11)
214
237


In [98]:
df_VPA_dosage.to_excel(project_path+'/data/processed_data/df_2.1.4.1_计算日剂量.xlsx')

#### 提取用药日期

In [99]:
# 临时用药的结束时间=开始时间
aaa=df_VPA_dosage[df_VPA_dosage['end_datetime'].isnull()]
bbb=df_VPA_dosage[df_VPA_dosage['end_datetime'].notnull()]
aaa['end_datetime']=aaa['start_datetime']

In [100]:
aaa

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量
5276,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2021-03-29 19:14:29,2021-03-29 19:14:29,核实,0.50
5277,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-03-31 09:09:50,2021-03-31 09:09:50,核实,1.00
5289,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-07 09:10:18,2021-04-07 09:10:18,核实,0.75
5352,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-09 19:49:24,2021-04-09 19:49:24,核实,0.75
8053,1173560,13397293,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-07-21 16:21:42,2021-07-21 16:21:42,核实,1.00
...,...,...,...,...,...,...,...,...,...,...,...
10182,9307105,14184132,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-09-30 10:29:44,2021-09-30 10:29:44,核实,1.00
10628,9424074,14701280,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-11-15 18:44:04,2021-11-15 18:44:04,核实,1.00
10777,9442416,14712892,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2021-11-17 10:47:51,2021-11-17 10:47:51,核实,0.50
10889,9442416,14712892,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-11-19 10:31:22,2021-11-19 10:31:22,核实,1.00


In [101]:
df_VPA_date=pd.concat([aaa,bbb],axis=0)
# 排序
df_VPA_date=df_VPA_date.sort_values(by=['patient_id','case_no','start_datetime'])
df_VPA_date=df_VPA_date.reset_index(drop=True)

In [102]:
print(df_VPA_date.shape)
print(df_VPA_date['patient_id'].nunique())
print(df_VPA_date['case_no'].nunique())

(767, 11)
214
237


In [103]:
# 提取丙戊酸用药时间的年月日
df_VPA_date['start_date']=df_VPA_date['start_datetime'].apply(lambda x: str(x).split(' ')[0])
df_VPA_date['end_date']=df_VPA_date['end_datetime'].apply(lambda x: str(x).split(' ')[0])

In [104]:
df_VPA_date.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date
0,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2021-03-29 19:14:29,2021-03-29 19:14:29,核实,0.50,2021-03-29,2021-03-29
1,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-03-31 09:09:50,2021-03-31 09:09:50,核实,1.00,2021-03-31,2021-03-31
2,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-07 09:10:18,2021-04-07 09:10:18,核实,0.75,2021-04-07,2021-04-07
3,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-09 19:49:24,2021-04-09 19:49:24,核实,0.75,2021-04-09,2021-04-09
4,1173560,13397293,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-07-21 16:21:42,2021-07-21 16:21:42,核实,1.00,2021-07-21,2021-07-21


In [105]:
df_VPA_date.to_excel(project_path+'/data/processed_data/df_2.1.3.2_提取年月日.xlsx')

#### 津源参考代码

In [106]:
# #对环孢素进行处理只包含注射液
# df = pd.read_excel(r'drug_2088_csa_injection_end.xls')

# df = df[['PATIENT_ID', 'DOSAGE_all','START_DATETIME', 'END_DATETIME']]
# df['PATIENT_ID'] = df['PATIENT_ID'].apply(lambda x: str(x))
# df['START_DATETIME'] = pd.to_datetime(df['START_DATETIME']).dt.strftime('%Y-%m-%d')
# df['END_DATETIME'] = pd.to_datetime(df['END_DATETIME']).dt.strftime('%Y-%m-%d')
# #%%
# import pandas as pd
# df['ll'] = df['DOSAGE_all']
# for i in range(len(df)):
#     df['ll'][i] = list(pd.date_range(start=df['START_DATETIME'][i], end=df['END_DATETIME'][i]))
    
# df['id_dose'] = df['PATIENT_ID'].map(str) + "_" + df['DOSAGE_all'].map(str)
# #%%
# df_1 = df[['id_dose','ll']]





# #%%
# def explode1(df, col):
#     dftem1 = pd.DataFrame()
#     for i in range(df.shape[0]):
#         dftem = df.iloc[i].to_frame().T
#         if isinstance(df.iloc[i][col], list):
#             if len(df.iloc[i][col]) > 0:
#                 for j in range(len(df.iloc[i][col])):
#                     dftem[col + "_"] = df.iloc[i][col][j]
#                     dftem1 = pd.concat([dftem1, dftem])
#             else:
#                 dftem1 = pd.concat([dftem1, dftem])
#         else:
#             dftem1 = pd.concat([dftem1, dftem])
#     del dftem1[col]
#     dftem1.rename(columns={col + "_": col}, inplace=True)
#     return dftem1

# #%%

# drug_2088_csa_injection_end_expand = explode1(df_1, 'll') 
# drug_2088_csa_injection_end_expand['PATIENT_ID'] = drug_2088_csa_injection_end_expand['id_dose'].apply(lambda x: x.split('_')[0])
# drug_2088_csa_injection_end_expand['DOSAGE'] = drug_2088_csa_injection_end_expand['id_dose'].apply(lambda x: x.split('_')[1])

# ll = drug_2088_csa_injection_end_expand
# drug_2088_csa_injection_end_expand1 = ll[[ 'PATIENT_ID', 'DOSAGE', 'll']].drop_duplicates(subset=['PATIENT_ID','ll'], keep='last')


#### 拆分长期医嘱和临时医嘱

In [108]:
# 打长期医嘱标签
df_VPA_date['long_d_order']=df_VPA_date['statusdesc'].apply(lambda x: 1 if re.match('停止',x) else 0)

In [109]:
# 长期医嘱和临时医嘱分开
drug_temp = df_VPA_date[df_VPA_date['long_d_order']==0]
drug_long = df_VPA_date[df_VPA_date['long_d_order']==1]

In [110]:
print(drug_temp.shape)
print(drug_long.shape)

(460, 14)
(307, 14)


In [111]:
drug_temp.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,long_d_order
0,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2021-03-29 19:14:29,2021-03-29 19:14:29,核实,0.50,2021-03-29,2021-03-29,0
1,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-03-31 09:09:50,2021-03-31 09:09:50,核实,1.00,2021-03-31,2021-03-31,0
2,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-07 09:10:18,2021-04-07 09:10:18,核实,0.75,2021-04-07,2021-04-07,0
3,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-09 19:49:24,2021-04-09 19:49:24,核实,0.75,2021-04-09,2021-04-09,0
4,1173560,13397293,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-07-21 16:21:42,2021-07-21 16:21:42,核实,1.00,2021-07-21,2021-07-21,0


In [112]:
# 查看重复数据
# drug_temp[drug_temp.duplicated(subset=['case_no','dosage','frequency','start_datetime','end_datetime','long_d_order'])]

In [113]:
# 删除重复数据
drug_temp=drug_temp.drop_duplicates(subset=['case_no','dosage','frequency','start_datetime','end_datetime','long_d_order'])
drug_long=drug_long.drop_duplicates(subset=['case_no','dosage','frequency','start_datetime','end_datetime','long_d_order'])

In [115]:
print(drug_temp.shape)
print(drug_long.shape)

(460, 14)
(307, 14)


#### 处理临时医嘱

In [119]:
# # 打临时时间标签，上午为0，下午为1
# drug_temp['用药上下午标签']=drug_temp['start_datetime'].apply(lambda x: 0 if x.hour <13 else 1)

In [128]:
# 人工处理一天多次临时用药，删除或合并，每天只有一条临时用药
'''
1/日-1/晚、1/早-1/晚、1/午-1/晚等，相加
1/日-ONCE、2/日-2/日、2/日-1/日，剂量相同-删除；剂量不同-相加
'''

In [121]:
drug_temp.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,long_d_order,用药上下午标签
0,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2021-03-29 19:14:29,2021-03-29 19:14:29,核实,0.50,2021-03-29,2021-03-29,0,1
1,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-03-31 09:09:50,2021-03-31 09:09:50,核实,1.00,2021-03-31,2021-03-31,0,0
2,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-07 09:10:18,2021-04-07 09:10:18,核实,0.75,2021-04-07,2021-04-07,0,0
3,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-09 19:49:24,2021-04-09 19:49:24,核实,0.75,2021-04-09,2021-04-09,0,1
4,1173560,13397293,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-07-21 16:21:42,2021-07-21 16:21:42,核实,1.00,2021-07-21,2021-07-21,0,1


In [124]:
# 调整临时医嘱的日剂量， 根据开药时间
def dosage_temp(x):
    if (x['frequency']==2)&(x['小时_标签']>=13):
        return x['日剂量']/2
    elif (x['frequency']==3):
        if (x['小时_标签']>=8) & (x['小时_标签']<12):
            return x['日剂量']*2/3
        elif x['小时_标签']>=12:
            return x['日剂量']/3
    else:
        return x['日剂量']
    
# 小时标签用来 标记临时医嘱的时间，根据 frequency进行 日剂量的更改
# 例如： 一日三次， 则 按照 12点和6点划分， 12-6之间是 2/3 
drug_temp['小时_标签'] = drug_temp['start_datetime'].dt.hour

drug_temp['日剂量'] = drug_temp.apply(dosage_temp,axis=1)
drug_temp = drug_temp.drop(columns='小时_标签')

In [123]:
drug_temp['start_datetime'].dt.hour

0      19
1       9
2       9
3      19
4      16
       ..
762    10
763    18
764    10
765    10
766    11
Name: start_datetime, Length: 460, dtype: int64

In [129]:
drug_temp.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,long_d_order,用药上下午标签
0,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2021-03-29 19:14:29,2021-03-29 19:14:29,核实,0.50,2021-03-29,2021-03-29,0,1
1,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-03-31 09:09:50,2021-03-31 09:09:50,核实,1.00,2021-03-31,2021-03-31,0,0
2,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-07 09:10:18,2021-04-07 09:10:18,核实,0.75,2021-04-07,2021-04-07,0,0
3,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-09 19:49:24,2021-04-09 19:49:24,核实,0.75,2021-04-09,2021-04-09,0,1
4,1173560,13397293,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-07-21 16:21:42,2021-07-21 16:21:42,核实,0.50,2021-07-21,2021-07-21,0,1


In [131]:
print(drug_temp.shape)
print(drug_temp.patient_id.nunique())
print(drug_temp.case_no.nunique())

(460, 15)
205
227


In [132]:
drug_temp.to_excel(project_path+'/data/processed_data/df_2.1.3.5_处理临时医嘱.xlsx')

#### 处理长期医嘱

In [ ]:
# 人工处理交叉的长期用药，合并或修改时间，处理成不存在用药期间交叉的长期用药
'''
1/日-1/晚，相加
1/日-1/晚、1/早-1/晚、1/午-1/晚等，相加
1/日-ONCE、2/日-2/日、2/日-1/日，剂量相同-删除；剂量不同-相加
如果上一次用药的结束日期=下一次用药的起始日期：
上 上，下一次日剂量
上、下；下 上；下 下，上一次日剂量*1/2+下一次日剂量*1/2
'''

In [134]:
# 日期分裂函数
def explode1(df, col):
    dftem1 = pd.DataFrame()
    for i in range(df.shape[0]):
        dftem = df.iloc[i].to_frame().T
        if isinstance(df.iloc[i][col], list):
            if len(df.iloc[i][col]) > 0:
                for j in range(len(df.iloc[i][col])):
                    dftem[col + "_"] = df.iloc[i][col][j]
                    dftem1 = pd.concat([dftem1, dftem])
            else:
                dftem1 = pd.concat([dftem1, dftem])
        else:
            dftem1 = pd.concat([dftem1, dftem])
    del dftem1[col]
    dftem1.rename(columns={col + "_": col}, inplace=True)
    return dftem1

In [141]:
# 分裂长期医嘱日期
drug_long['分裂日期']=''
drug_long=drug_long.reset_index(drop=True)
for i in range(len(drug_long)):
    drug_long['分裂日期'][i] = list(pd.date_range(start=drug_long['start_date'][i],end=drug_long['end_date'][i]))
    
drug_long=explode1(drug_long,'分裂日期')

In [147]:
drug_long.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,long_d_order,分裂日期
0,1285315,4922567,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2018-11-10 15:39:01,2018-11-30 08:15:00,停止,1,2018-11-10,2018-11-30,1,2018-11-10
0,1285315,4922567,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2018-11-10 15:39:01,2018-11-30 08:15:00,停止,1,2018-11-10,2018-11-30,1,2018-11-11
0,1285315,4922567,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2018-11-10 15:39:01,2018-11-30 08:15:00,停止,1,2018-11-10,2018-11-30,1,2018-11-12
0,1285315,4922567,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2018-11-10 15:39:01,2018-11-30 08:15:00,停止,1,2018-11-10,2018-11-30,1,2018-11-13
0,1285315,4922567,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2018-11-10 15:39:01,2018-11-30 08:15:00,停止,1,2018-11-10,2018-11-30,1,2018-11-14


In [146]:
drug_long.to_excel('./data/processed_data/df_2.1.3.6_拆分长期医嘱.xlsx')

In [148]:
# 调整上一次结束日期=下一次起始的日剂量
temp_list=[]
for i in drug_long.case_no.unique():
    temp=drug_long[drug_long['case_no']==i]
    for j in range(temp.shape[0]):
        if temp.loc[j,'分裂日期']==temp.loc[j+1,'分裂日期']:
            pass

KeyError: 1

In [137]:
# 分裂长期医嘱日期
drug_long=explode1(drug_long,'分裂日期')

drug_long['分裂日期']=drug_long['分裂日期'].apply(lambda x: str(x).split(' ')[0])

drug_long['起始标签'] = drug_long.apply(lambda x: '起始' if x['分裂日期']==x['start_date'] else 
                                                    '结尾' if x['分裂日期']==x['end_date'] else 
                                                    '期间', axis=1)

KeyError: '分裂日期'

In [ ]:
drug_long.to_excel(project_path+'/data/processed_data/df_2.1.3.6_处理长期医嘱.xlsx')

#### 处理临时医嘱

In [115]:
# 打临时时间标签，上午为0，下午为1
drug_temp['用药上下午标签']=drug_temp['start_datetime'].apply(lambda x: 0 if x.hour <13 else 1)

In [116]:
drug_temp_all = drug_temp.drop_duplicates(subset=['patient_id','start_date'],keep='first')

In [117]:
# ======= 单独对‘临时医嘱’处理 =======
drug_temp_all = drug_temp.drop_duplicates(subset=['patient_id','start_date'],keep='first')
drug_temp_1 = drug_temp.drop_duplicates(subset=['patient_id','start_date','用药上下午标签'],keep='first')
drug_temp_group = drug_temp_1.groupby(['patient_id','start_date'])['日剂量'].sum().reset_index()   # 同一天的临时医嘱相加，保证每天一条临时医嘱

# 拼接完整数据
drug_temp_result = pd.merge(drug_temp_all.drop(columns=['日剂量','用药上下午标签']),drug_temp_group,on=['patient_id','start_date'],how='inner')

In [118]:
print(drug_temp_result.shape)

(460, 14)


In [119]:
# 计算临时医嘱的 日剂量， 根据开药时间
def dosage_temp(x):
    if (x['frequency']==2)&(x['小时_标签']>=13):
        return x['日剂量']/2
    elif (x['frequency']==3):
        if (x['小时_标签']>=8) & (x['小时_标签']<12):
            return x['日剂量']*2/3
        elif x['小时_标签']>=12:
            return x['日剂量']/3
    else:
        return x['日剂量']
    
# 小时标签用来 标记临时医嘱的时间，根据 frequency进行 日剂量的更改
# 例如： 一日三次， 则 按照 12点和6点划分， 12-6之间是 2/3 
drug_temp_result['小时_标签'] = drug_temp_result['start_datetime'].dt.hour

drug_temp_result['日剂量'] = drug_temp_result.apply(dosage_temp,axis=1)
drug_temp_result = drug_temp_result.drop(columns='小时_标签')

In [120]:
drug_temp_all.drop(columns=['日剂量','用药上下午标签'])

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,start_date,end_date,long_d_order
0,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2021-03-29 19:14:29,2021-03-29 19:14:29,核实,2021-03-29,2021-03-29,0
1,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-03-31 09:09:50,2021-03-31 09:09:50,核实,2021-03-31,2021-03-31,0
2,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-07 09:10:18,2021-04-07 09:10:18,核实,2021-04-07,2021-04-07,0
3,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-09 19:49:24,2021-04-09 19:49:24,核实,2021-04-09,2021-04-09,0
4,1173560,13397293,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-07-21 16:21:42,2021-07-21 16:21:42,核实,2021-07-21,2021-07-21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,9307105,14184132,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-09-30 10:29:44,2021-09-30 10:29:44,核实,2021-09-30,2021-09-30,0
763,9424074,14701280,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-11-15 18:44:04,2021-11-15 18:44:04,核实,2021-11-15,2021-11-15,0
764,9442416,14712892,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2021-11-17 10:47:51,2021-11-17 10:47:51,核实,2021-11-17,2021-11-17,0
765,9442416,14712892,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-11-19 10:31:22,2021-11-19 10:31:22,核实,2021-11-19,2021-11-19,0


In [121]:
print(drug_temp_result.shape)
print(drug_temp_result['patient_id'].nunique())
print(drug_temp_result['case_no'].nunique())

(460, 14)
205
227


In [122]:
drug_temp_result.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,start_date,end_date,long_d_order,日剂量
0,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2021-03-29 19:14:29,2021-03-29 19:14:29,核实,2021-03-29,2021-03-29,0,0.50
1,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-03-31 09:09:50,2021-03-31 09:09:50,核实,2021-03-31,2021-03-31,0,1.00
2,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-07 09:10:18,2021-04-07 09:10:18,核实,2021-04-07,2021-04-07,0,0.75
3,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-09 19:49:24,2021-04-09 19:49:24,核实,2021-04-09,2021-04-09,0,0.75
4,1173560,13397293,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-07-21 16:21:42,2021-07-21 16:21:42,核实,2021-07-21,2021-07-21,0,0.50


In [123]:
# 把‘临时医嘱’和‘长期医嘱’拼接
df_test_1 = pd.concat([drug_long,drug_temp_result],axis=0)
df_test_1 = df_test_1[drug_long.columns.tolist()]

# 拼接后重新排序
drug_vancomycin_4 = df_test_1.groupby(['case_no']).apply(
    lambda _df: _df.sort_values(by=['start_datetime'],ascending=1)).reset_index(drop=True)

In [124]:
drug_vancomycin_4.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,long_d_order
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1
1,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-24 10:37:31,2020-09-04 11:38:00,停止,1.0,2020-08-24,2020-09-04,1
2,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-25 17:31:34,2020-08-25 17:31:34,核实,0.5,2020-08-25,2020-08-25,0
3,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-09-04 11:38:55,2020-09-04 11:38:55,核实,1.0,2020-09-04,2020-09-04,0
4,547307,10005574,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2020-08-16 16:41:46,2020-08-31 11:15:00,停止,1.0,2020-08-16,2020-08-31,1


In [125]:
drug_vancomycin_4.to_excel(project_path+'/data/processed_data/df_2.1.4.5_处理临时医嘱.xlsx')

#### 分裂日期

In [126]:
# 将时间拆开
def explode1(df, col):
    dftem1 = pd.DataFrame()
    for i in range(df.shape[0]):
        dftem = df.iloc[i].to_frame().T
        if isinstance(df.iloc[i][col], list):
            if len(df.iloc[i][col]) > 0:
                for j in range(len(df.iloc[i][col])):
                    dftem[col + "_"] = df.iloc[i][col][j]
                    dftem1 = pd.concat([dftem1, dftem])
            else:
                dftem1 = pd.concat([dftem1, dftem])
        else:
            dftem1 = pd.concat([dftem1, dftem])
    del dftem1[col]
    dftem1.rename(columns={col + "_": col}, inplace=True)
    return dftem1

In [127]:
drug_vancomycin_4.shape

(767, 14)

In [128]:
# 生成 分裂日期
drug_vancomycin_4['分裂日期'] = ''
for i in range(len(drug_vancomycin_4)):
    drug_vancomycin_4['分裂日期'][i] = list(pd.date_range(start=drug_vancomycin_4['start_date'][i], 
                                                      end=drug_vancomycin_4['end_date'][i]))

##　＝＝　把时间List拆开　＝＝    
drug_vancomycin_5 = explode1(drug_vancomycin_4,'分裂日期')
drug_vancomycin_5['分裂日期']=drug_vancomycin_5['分裂日期'].apply(lambda x: str(x).split(' ')[0])

drug_vancomycin_5['起始标签'] = drug_vancomycin_5.apply(lambda x: '起始' if x['分裂日期']==x['start_date'] else 
                                                                '结尾' if x['分裂日期']==x['end_date'] else 
                                                                '期间', axis=1)

In [129]:
drug_vancomycin_5.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,long_d_order,分裂日期,起始标签
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-15,起始
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-16,期间
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-17,期间
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-18,期间
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-19,期间


In [130]:
print(drug_vancomycin_5.shape)
print(drug_vancomycin_5['patient_id'].nunique())
print(drug_vancomycin_5['case_no'].nunique())

(3541, 16)
214
237


In [131]:
drug_vancomycin_5.to_excel(project_path+'/data/processed_data/df_2.1.4.6_分裂日期.xlsx')

#### 打时间标签

In [132]:
# 打日期标签
def flag_ttt(x):
    if (x['起始标签']!='结尾'):
        if x['start_datetime'].hour<13:
            return 0
        else:
            return 1
    else:
        if x['end_datetime'].hour<13:
            return 0
        else:
            return 1

In [133]:
# 上午是0，下午是1
# 起始和期间看 起始时间， 结尾看停止时间
drug_vancomycin_5['时间标签'] = drug_vancomycin_5.apply(lambda x: 0 if ((x['起始标签']!='结尾')and(x['start_datetime'].hour < 13)) else 1, axis=1)
drug_vancomycin_5['时间标签'] = drug_vancomycin_5.apply(flag_ttt,axis=1)

In [134]:
print(drug_vancomycin_5.shape)
print(drug_vancomycin_5['patient_id'].nunique())
print(drug_vancomycin_5['case_no'].nunique())

(3541, 17)
214
237


In [135]:
drug_vancomycin_5.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,long_d_order,分裂日期,起始标签,时间标签
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-15,起始,1
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-16,期间,1
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-17,期间,1
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-18,期间,1
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-19,期间,1


In [136]:
drug_vancomycin_5.to_excel(project_path+'/data/processed_data/df_2.1.4.7_打时间标签.xlsx')

#### 调整日剂量

In [137]:
temp_l = []

for i in drug_vancomycin_5.case_no.unique():
    temp = drug_vancomycin_5[drug_vancomycin_5['case_no']==i].reset_index(drop=True)

    for j in temp.分裂日期.unique():

        temp_2 = temp[temp['分裂日期']==j]
        if 0 in temp_2['long_d_order'].unique():
            temp_long = temp_2[temp_2['long_d_order']==1].tail(1).reset_index(drop=True)
            temp_st = temp_2[temp_2['long_d_order']==0]
            temp_st['日剂量'] = temp_st['日剂量'].sum()
            temp_st = temp_st.tail(1).reset_index(drop=True)
            # 同一天，既有长期医嘱又有短期医嘱
            if len(temp_long)>0:
                flag = temp_long.起始标签.unique()[0]
                if flag == '期间':
                    temp_long.loc[0,'日剂量'] = temp_long.loc[0,'日剂量'] + temp_st.loc[0,'日剂量']                           
                    temp_l.append(temp_long)
                else:
                    temp_l.append(temp_st)
            # 该天全是短期医嘱
            else:
                temp_l.append(temp_st)
        # 该天全是长期医嘱
        else:
            temp_long = temp_2.tail(1).reset_index(drop=True)
            flag_l = temp_2.起始标签.unique().tolist()
            flag_time_l = temp_2.时间标签.unique().tolist()
            
            if ('结尾' in flag_l and '起始' in flag_l) or len(flag_time_l)>1:
                # 按剂量去重（防止上一次结束，与下一次开始是两种情况）                
                if (len(flag_time_l) == 1) and (temp_long.loc[0,'时间标签'] == 0):   
                    pass                                                         
                # 第一条结尾，第二条开始:   时间 上午 上午 取第二条(开始), 上下、下上、下下取mean
                # 按剂量去重（防止上一次结束，与下一次开始是两种情况）
                else:
                    temp_long.loc[0,'日剂量'] = temp_2.drop_duplicates(subset=['日剂量'],keep='last').日剂量.mean()
                temp_l.append(temp_long)
            else:
                flag = temp_long.loc[0,'起始标签']
                flag_time = temp_long.loc[0,'时间标签']
                # 开头：大于13点取半天，  结尾：小于13点取半天
                if flag=='起始' and flag_time==1:
                    temp_long.loc[0,'日剂量'] = temp_long.loc[0,'日剂量'] / 2
                if flag=='结尾' and flag_time==0:
                    temp_long.loc[0,'日剂量'] = temp_long.loc[0,'日剂量'] / 2

                temp_l.append(temp_long)

In [138]:
df_VPA_modification = pd.concat(temp_l,axis=0).reset_index(drop=True)
df_VPA_modification['日剂量']=df_VPA_modification['日剂量'].apply(lambda x: round(x,2))
del temp_l

In [139]:
# 设置patient_id，case_no字段数据数据格式，不然无法匹配id
df_VPA_modification['patient_id']=df_VPA_modification['patient_id'].astype('str')
df_VPA_modification['case_no']=df_VPA_modification['case_no'].astype('str')

In [140]:
print(df_VPA_modification.shape)
print(df_VPA_modification['patient_id'].nunique())
print(df_VPA_modification['case_no'].nunique())

(3188, 17)
214
237


In [141]:
df_VPA_modification.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,long_d_order,分裂日期,起始标签,时间标签
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.25,2020-08-15,2020-08-24,1,2020-08-15,起始,1
1,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.50,2020-08-15,2020-08-24,1,2020-08-16,期间,1
2,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.50,2020-08-15,2020-08-24,1,2020-08-17,期间,1
3,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.50,2020-08-15,2020-08-24,1,2020-08-18,期间,1
4,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.50,2020-08-15,2020-08-24,1,2020-08-19,期间,1


In [142]:
df_VPA_modification.to_excel(project_path+'/data/processed_data/df_2.1.4.8_调整日剂量.xlsx')

In [143]:
#### 用药稳态筛选

In [144]:
# df_VPA_steady=df_VPA_modification.copy()
# # 到稳态血药浓度（同一剂量至少连续服用3天)
# temp_list=[]
# for i in df_VPA_steady['case_no'].unique():
#     print(i)
#     temp=df_VPA_steady[df_VPA_steady['case_no']==i]
#     temp=temp.reset_index(drop=True)
#     j=0
#     while j < temp.shape[0]:
# #         print(j)
#         # 连续服药3天，tdm达到稳态。1.剂量连续3天；2. 时间连续3天
#         dosage_judge=temp.loc[j,'日剂量']    
#         # 1. 剂量判断
#         break_bool=False
#         for k in range(j+1,temp.shape[0]):
#             differ = (pd.to_datetime(temp.loc[k,'分裂日期'])-pd.to_datetime(temp.loc[k-1,'分裂日期'])).days
#             if dosage_judge != temp.loc[k,'日剂量'] or differ != 1:
#                 # 判断是否中断
#                 break_bool=True
#                 break
#         # 是否是最后一天
#         if k != temp.shape[0]-1:
#             # 是否是稳态
#             if k-1-j>=2:
#                 temp_list.append(temp.loc[j:k-1,])
#             j=k
#             continue
#         else:
#             # 最后一天是否中断
#             if break_bool:
#                 # 是否是稳态
#                 if k-1-j>=2:
#                     temp_list.append(temp.loc[j:k-1,])
#             else:
#                 if k-j >=2:
#                     temp_list.append(temp.loc[j:k,])
#             j=k+1

# # list转换为DataFrame
# df_VPA_steady = pd.concat(temp_list,axis=0).reset_index(drop=True)
# del temp_list

# print(df_VPA_steady.shape)
# print(df_VPA_steady['patient_id'].nunique())
# print(df_VPA_steady['case_no'].nunique())

# df_VPA_steady.head()

# df_VPA_steady.to_excel(project_path+'/data/processed_data/df_2.1.4.7_用药稳态筛选.xlsx')

## 排除标准

### 妊娠或哺乳期妇女

In [145]:
# 字段“尿妊娠试验”阳性为怀孕；“β人绒毛膜促性腺激素”在正常值范围内表明没有怀孕；
df_pregnant_1=df_test[df_test['project_name'].str.contains('尿妊娠试验')]
df_pregnant_1=df_pregnant_1[df_pregnant_1['test_result'].str.contains('阳性')]
df_pregnant_2=df_test[df_test['project_name'].str.contains('β人绒毛膜促性腺激素')]
df_pregnant_2=df_pregnant_2[df_pregnant_2['test_result'].astype('float')>5.3]

In [146]:
df_test_pregnant=pd.concat([df_pregnant_1,df_pregnant_2],axis=0)

In [147]:
df_test_pregnant

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym
19330,800005121742||C122||1,7565842,6917667,2019-07-13 11:34:00,双相情感障碍，目前为重度抑郁发作,血清,NaT,β人绒毛膜促性腺激素,107584.000,mIU/mL,N,绝经后(0.00-8.30) 绝经前(0.00-5.30)\n\r,βHCG


In [148]:
# 剔除妊娠妇女
df_VPA_pregnant=df_VPA_modification[~ df_VPA_modification['patient_id'].isin(df_test_pregnant.patient_id.unique())]

In [149]:
print(df_VPA_pregnant.shape)
print(df_VPA_pregnant['patient_id'].nunique())
print(df_VPA_pregnant['case_no'].nunique())

(3175, 17)
213
236


In [150]:
# 排除妊娠妇女
df_VPA_pregnant.to_excel(project_path+'/data/processed_data/df_2.2.1_排除妊娠妇女.xlsx')

### 排除数据严重缺失

In [151]:
# 数据严重缺失（如患者ID、主要研究结局等
df_VPA_pregnant.isnull().any()

patient_id        False
case_no           False
drug_name         False
amount            False
drug_spec         False
dosage            False
frequency         False
start_datetime    False
end_datetime      False
statusdesc        False
日剂量               False
start_date        False
end_date          False
long_d_order      False
分裂日期              False
起始标签              False
时间标签              False
dtype: bool

# 合并人口学特征

## 合并人口学信息

In [152]:
# 取 patient_id和case_no
df_popu=df_VPA_pregnant[['patient_id','case_no']]

In [153]:
# merge gender
df_popu=pd.merge(df_popu,df_patient_info,on=['patient_id'],how='left')

In [154]:
# merge age
df_age_simple=df_age[['case_no','age']]
df_popu=pd.merge(df_popu,df_age_simple,on=['case_no'],how='left')

In [155]:
# merge height
df_height_simple=df_height[['case_no','record_content']]
df_height_simple=df_height_simple.rename(columns={'record_content':'身高'})
df_height_simple['身高']=df_height_simple['身高'].astype('float')
df_popu=pd.merge(df_popu,df_height_simple,on=['case_no'],how='left')

In [156]:
# merge weight
df_weight_simple=df_weight[['case_no','record_content']]
df_weight_simple=df_weight_simple.rename(columns={'record_content':'体重'})
df_weight_simple['体重']=df_weight_simple['体重'].astype('float')
df_popu=pd.merge(df_popu,df_weight_simple,on=['case_no'],how='left')

In [157]:
df_weight_simple.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246 entries, 3600 to 19370
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   case_no  246 non-null    object 
 1   体重       246 non-null    float64
dtypes: float64(1), object(1)
memory usage: 5.8+ KB


In [158]:
df_popu.head()

,patient_id,case_no,gender,race,birth_year,age,身高,体重
0,6873870,10001302,0,汉族,1992-11-06,28,153.0,52.0
1,6873870,10001302,0,汉族,1992-11-06,28,153.0,52.0
2,6873870,10001302,0,汉族,1992-11-06,28,153.0,52.0
3,6873870,10001302,0,汉族,1992-11-06,28,153.0,52.0
4,6873870,10001302,0,汉族,1992-11-06,28,153.0,52.0


In [159]:
print(df_popu.shape)
print(df_popu['patient_id'].nunique())

(4084, 8)
213


In [160]:
# 删除重复数据
df_popu=df_popu.drop_duplicates(['patient_id','case_no','gender'])

In [161]:
print(df_popu.shape)
print(df_popu['patient_id'].nunique())
print(df_popu.case_no.nunique())

(236, 8)
213
236


In [162]:
df_popu.to_excel(project_path+'/data/processed_data/df_3.1_popu.xlsx')

## 计算BMI

In [163]:
BMI_list=[round(x*10000/y/y,2) for (x,y) in zip(df_popu['体重'].astype('float').tolist(),df_popu['身高'].astype('float').tolist())]

In [164]:
print(len(BMI_list))
print(df_popu.shape)

236
(236, 8)


In [165]:
df_popu['BMI']=np.array(BMI_list)

In [166]:
df_popu

,patient_id,case_no,gender,race,birth_year,age,身高,体重,BMI
0,6873870,10001302,0,汉族,1992-11-06,28,153.0,52.0,22.21
21,547307,10005574,0,汉族,1995-05-13,25,162.0,52.0,19.81
37,6949939,10019847,0,汉族,1993-11-22,27,160.0,45.0,17.58
55,7765586,10024628,0,汉族,2003-12-08,16,168.0,82.0,29.05
67,369329,10028911,0,汉族,1987-11-03,33,159.0,67.0,26.50
...,...,...,...,...,...,...,...,...,...
4018,2583650,9818116,0,汉族,1961-10-27,59,158.0,76.0,30.44
4026,7411504,9835190,0,汉族,2003-10-15,17,163.0,57.0,21.45
4039,8334957,9890262,1,汉族,2006-07-25,14,166.0,49.0,17.78
4054,6865481,9910592,1,汉族,2000-11-06,20,183.0,NaN,NaN


## 并入人口学特征

In [167]:
df_popu=df_popu[['case_no','gender','age','身高','体重','BMI']]

In [168]:
print(df_VPA_pregnant.shape)
print(df_VPA_pregnant['patient_id'].nunique())
print(df_VPA_pregnant['case_no'].nunique())

(3175, 17)
213
236


In [169]:
df_VPA_popu=pd.merge(df_VPA_pregnant,df_popu,on=['case_no'],how='left')

In [170]:
print(df_VPA_popu.shape)
print(df_VPA_popu['patient_id'].nunique())
print(df_VPA_popu['case_no'].nunique())

(3175, 22)
213
236


In [171]:
# 合并人口学特征
df_VPA_popu.to_excel(project_path+'/data/processed_data/df_3.3_并入人口学特征.xlsx')

# 基础疾病

In [172]:
# 将高血压、糖尿病、脑器质疾病、肝炎、肾炎合并为基础疾病一项

In [173]:
# 高血压、糖尿病
df_VPA_history=df_VPA_popu.copy()
# 提取高血压
df_EH=df_diagnostic_record[df_diagnostic_record['diagnostic_content'].str.contains('高血压')]
# 提取糖尿病
df_DM=df_diagnostic_record[df_diagnostic_record['diagnostic_content'].str.contains('糖尿病')]
# 脑器质疾病：脑退行性改变（痴呆）、脑血管病（脑梗塞/缺血性卒中/中风、脑出血）、颅内感染、脑外伤、脑肿瘤、癫痫
# 提取脑器质疾病brain organic mental disorders
df_bomd=df_diagnostic_record[(df_diagnostic_record['diagnostic_content'].str.contains('器质'))&
                            df_diagnostic_record['diagnostic_content'].str.contains('精神')]
# 提取肝炎
df_HCA=df_diagnostic_record[df_diagnostic_record['diagnostic_content'].str.contains('肝炎')]
# 提取肾病，不包括肾结石
df_NEP=df_diagnostic_record[(df_diagnostic_record['diagnostic_content'].str.contains('肾'))&
                           (~ df_diagnostic_record['diagnostic_content'].str.contains('肾结石'))]

In [174]:
df_basic_disease=pd.concat([df_EH,df_DM],axis=0)
df_basic_disease=pd.concat([df_basic_disease,df_bomd],axis=0)
df_basic_disease=pd.concat([df_basic_disease,df_HCA],axis=0)
df_basic_disease=pd.concat([df_basic_disease,df_NEP],axis=0)

In [175]:
df_VPA_history['基础疾病']=df_VPA_history['patient_id'].apply(lambda x: 1 if x in df_basic_disease.patient_id.unique() else 0)

In [176]:
print(df_VPA_history.shape)
print(df_VPA_history['patient_id'].nunique())
print(df_VPA_history['case_no'].nunique())

(3175, 23)
213
236


In [177]:
df_VPA_history.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,...,long_d_order,分裂日期,起始标签,时间标签,gender,age,身高,体重,BMI,基础疾病
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,...,1,2020-08-15,起始,1,0,28,153.0,52.0,22.21,0
1,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,...,1,2020-08-16,期间,1,0,28,153.0,52.0,22.21,0
2,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,...,1,2020-08-17,期间,1,0,28,153.0,52.0,22.21,0
3,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,...,1,2020-08-18,期间,1,0,28,153.0,52.0,22.21,0
4,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,...,1,2020-08-19,期间,1,0,28,153.0,52.0,22.21,0


In [178]:
# 保存纳入标准数据
df_VPA_history.to_excel(project_path+'/data/processed_data/df_4.3_并入基础疾病.xlsx')

# 并入出入院时间

In [179]:
df_inp_record_simple=df_inp_record[['case_no','adm_date','dis_date']]

In [180]:
# 并入出入院时间
df_VPA_inp_time=pd.merge(df_VPA_history,df_inp_record_simple,on='case_no',how='left')

In [181]:
print(df_VPA_inp_time.shape)
print(df_VPA_inp_time['patient_id'].nunique())
print(df_VPA_inp_time['case_no'].nunique())

(3175, 25)
213
236


## 补充住院时间

In [182]:
# 补充住院时间
aaa=df_VPA_inp_time[df_VPA_inp_time['adm_date'].isnull()]
bbb=df_VPA_inp_time[df_VPA_inp_time['adm_date'].notnull()]
aaa['adm_date']=aaa['start_datetime']
df_VPA_inp_time=pd.concat([aaa,bbb],axis=0)
df_VPA_inp_time=df_VPA_inp_time.sort_values(by=['patient_id','case_no','分裂日期'])
df_VPA_inp_time=df_VPA_inp_time.reset_index(drop=True)

In [183]:
# 补充出院时间
aaa=df_VPA_inp_time[df_VPA_inp_time['dis_date'].isnull()]
bbb=df_VPA_inp_time[df_VPA_inp_time['dis_date'].notnull()]
aaa['dis_date']=aaa['end_datetime']
df_VPA_inp_time=pd.concat([aaa,bbb],axis=0)
df_VPA_inp_time=df_VPA_inp_time.sort_values(by=['patient_id','case_no','分裂日期'])
df_VPA_inp_time=df_VPA_inp_time.reset_index(drop=True)

In [184]:
df_VPA_inp_time.to_excel(project_path+'/data/processed_data/df_5.1_补充住院时间.xlsx')

## 计算住院时长

In [185]:
# 计算住院时长
df_VPA_inp_time['住院时长']=(pd.to_datetime(df_VPA_inp_time['dis_date']) - pd.to_datetime(df_VPA_inp_time['adm_date']))
df_VPA_inp_time['住院时长']=df_VPA_inp_time['住院时长'].apply(lambda x:x.days+1)

In [186]:
print(df_VPA_inp_time.shape)
print(df_VPA_inp_time['patient_id'].nunique())
print(df_VPA_inp_time['case_no'].nunique())

(3175, 26)
213
236


In [187]:
df_VPA_inp_time.to_excel(project_path+'/data/processed_data/df_5.2_计算住院时长.xlsx')

# 并入安全性指标

## 提取安全性指标

In [188]:
# 肝功能异常：丙氨酸氨基转移酶(0-40U/L)--24、天门冬氨酸氨基转移酶(0-35U/L)、转氨酶比值(1.0-1.5)、总蛋白(65.0-85.0)、白蛋白(40.0-55.0)、
        # 球蛋白(20.0-40.0)、白/球比值(1.2-2.4)（有一个指标不正常就算肝功能异常）
# 皮疹：复方酮氯乳膏、莫米松乳膏（在治疗后使用，算为不良反应）
# 恶心：多潘立酮、甲氧氯普胺（在治疗后使用，算为不良反应）
# 血小板减少：血小板计数<100x109（在治疗后）

### 肝功能异常

#### 丙氨酸氨基转移酶ALT

In [189]:
df_test_ALT=df_test[df_test['project_name'].str.contains('丙氨酸氨基转移酶')]

# 患者限定
df_test_ALT=df_test_ALT[df_test_ALT.case_no.isin(df_VPA_inp_time.case_no.unique())]

# 指标0-1转换
df_test_ALT['肝功_label']=df_test_ALT['test_result'].apply(lambda x: 0 if float(x)<=40 else 1)

In [190]:
df_test_ALT.head()

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym,肝功_label
2345,800001934594||B005||1,365567,4028318,2018-07-18 13:31:00,双相情感障碍，目前为混合性发作,血清,2018-07-18 07:25:07,丙氨酸氨基转移酶,17,U/L,N,9-50,ALT,0
2532,800001953715||B005||1,1819998,4045163,2018-07-20 11:02:00,双相情感障碍，目前为混合性发作,血清,2018-07-20 07:40:33,丙氨酸氨基转移酶,42,U/L,H,7-40,ALT,1
2665,800001979565||B005||1,7412218,4069083,2018-07-24 11:48:00,双相情感障碍，目前为伴有精神病性症状的躁狂发作,血清,2018-07-24 07:05:36,丙氨酸氨基转移酶,24,U/L,N,7-40,ALT,0
2794,800002012932||B005||1,7459164,4103620,2018-07-27 11:22:00,双相情感障碍，目前为混合性发作,血清,2018-07-27 07:40:37,丙氨酸氨基转移酶,9,U/L,N,9-50,ALT,0
2901,800002023150||B005||1,7461862,4112018,2018-07-28 11:40:00,双相情感障碍目前为不伴精神病的重度抑郁发作,血清,2018-07-28 07:38:51,丙氨酸氨基转移酶,7,U/L,L,9-50,ALT,0


#### 天门冬氨酸氨基转移酶AST

In [191]:
df_test_AST=df_test[df_test['project_name'].str.contains('天门冬氨酸氨基转移酶')]

# 患者限定
df_test_AST=df_test_AST[df_test_AST.case_no.isin(df_VPA_inp_time.case_no.unique())]

# 指标0-1转换
df_test_AST['肝功_label']=df_test_AST['test_result'].apply(lambda x: 0 if float(x)<=35 else 1)

In [192]:
df_test_AST.head()

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym,肝功_label
2340,800001934594||B002||1,365567,4028318,2018-07-18 13:31:00,双相情感障碍，目前为混合性发作,血清,2018-07-18 07:25:07,天门冬氨酸氨基转移酶,18,U/L,N,15-40,AST,0
2346,800001934594||B005||1,365567,4028318,2018-07-18 13:31:00,双相情感障碍，目前为混合性发作,血清,2018-07-18 07:25:07,天门冬氨酸氨基转移酶,18,U/L,N,15-40,AST,0
2527,800001953715||B002||1,1819998,4045163,2018-07-20 11:02:00,双相情感障碍，目前为混合性发作,血清,2018-07-20 07:40:33,天门冬氨酸氨基转移酶,30,U/L,N,13-35,AST,0
2533,800001953715||B005||1,1819998,4045163,2018-07-20 11:02:00,双相情感障碍，目前为混合性发作,血清,2018-07-20 07:40:33,天门冬氨酸氨基转移酶,30,U/L,N,13-35,AST,0
2660,800001979565||B002||1,7412218,4069083,2018-07-24 11:48:00,双相情感障碍，目前为伴有精神病性症状的躁狂发作,血清,2018-07-24 07:05:36,天门冬氨酸氨基转移酶,32,U/L,N,13-35,AST,0


#### 转氨酶比值

In [193]:
df_test_zhuananmei=df_test[df_test['project_name'].str.contains('转氨酶比值')]
# 患者限定
df_test_zhuananmei=df_test_zhuananmei[df_test_zhuananmei.case_no.isin(df_VPA_inp_time.case_no.unique())]
# 指标0-1转换
df_test_zhuananmei['肝功_label']=df_test_zhuananmei['test_result'].apply(lambda x: 0 if 1.0<=float(x)<=1.5 else 1)

In [194]:
df_test_zhuananmei.head()

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym,肝功_label
2347,800001934594||B005||1,365567,4028318,2018-07-18 13:31:00,双相情感障碍，目前为混合性发作,血清,2018-07-18 07:25:07,转氨酶比值,1.1,None,N,1.0-1.5,AST/ALT,0
2534,800001953715||B005||1,1819998,4045163,2018-07-20 11:02:00,双相情感障碍，目前为混合性发作,血清,2018-07-20 07:40:33,转氨酶比值,0.7,None,L,1.0-1.5,AST/ALT,1
2667,800001979565||B005||1,7412218,4069083,2018-07-24 11:48:00,双相情感障碍，目前为伴有精神病性症状的躁狂发作,血清,2018-07-24 07:05:36,转氨酶比值,1.3,None,N,1.0-1.5,AST/ALT,0
2796,800002012932||B005||1,7459164,4103620,2018-07-27 11:22:00,双相情感障碍，目前为混合性发作,血清,2018-07-27 07:40:37,转氨酶比值,1.6,None,H,1.0-1.5,AST/ALT,1
2903,800002023150||B005||1,7461862,4112018,2018-07-28 11:40:00,双相情感障碍目前为不伴精神病的重度抑郁发作,血清,2018-07-28 07:38:51,转氨酶比值,1.4,None,N,1.0-1.5,AST/ALT,0


#### 总蛋白TP

In [195]:
df_test_TP=df_test[df_test['project_name']=='总蛋白']
# 患者限定
df_test_TP=df_test_TP[df_test_TP.case_no.isin(df_VPA_inp_time.case_no.unique())]
# 指标0-1转换
df_test_TP['肝功_label']=df_test_TP['test_result'].apply(lambda x: 0 if 65<=float(x)<=85 else 1)

In [196]:
df_test_TP.head()

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym,肝功_label
2348,800001934594||B005||1,365567,4028318,2018-07-18 13:31:00,双相情感障碍，目前为混合性发作,血清,2018-07-18 07:25:07,总蛋白,66.3,g/L,N,65.0-85.0,TP,0
2535,800001953715||B005||1,1819998,4045163,2018-07-20 11:02:00,双相情感障碍，目前为混合性发作,血清,2018-07-20 07:40:33,总蛋白,62.1,g/L,L,65.0-85.0,TP,1
2668,800001979565||B005||1,7412218,4069083,2018-07-24 11:48:00,双相情感障碍，目前为伴有精神病性症状的躁狂发作,血清,2018-07-24 07:05:36,总蛋白,76.2,g/L,N,65.0-85.0,TP,0
2797,800002012932||B005||1,7459164,4103620,2018-07-27 11:22:00,双相情感障碍，目前为混合性发作,血清,2018-07-27 07:40:37,总蛋白,78.9,g/L,N,65.0-85.0,TP,0
2904,800002023150||B005||1,7461862,4112018,2018-07-28 11:40:00,双相情感障碍目前为不伴精神病的重度抑郁发作,血清,2018-07-28 07:38:51,总蛋白,73.8,g/L,N,65.0-85.0,TP,0


#### 白蛋白ALB

In [197]:
df_test_ALB=df_test[df_test['project_name']=='白蛋白']
# 患者限定
df_test_ALB=df_test_ALB[df_test_ALB.case_no.isin(df_VPA_inp_time.case_no.unique())]
# 指标0-1转换
df_test_ALB['肝功_label']=df_test_ALB['test_result'].apply(lambda x: 0 if 40<=float(x)<=55 else 1)

In [198]:
df_test_ALB.head()

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym,肝功_label
2349,800001934594||B005||1,365567,4028318,2018-07-18 13:31:00,双相情感障碍，目前为混合性发作,血清,2018-07-18 07:25:07,白蛋白,41.4,g/L,N,40.0-55.0,ALB,0
2536,800001953715||B005||1,1819998,4045163,2018-07-20 11:02:00,双相情感障碍，目前为混合性发作,血清,2018-07-20 07:40:33,白蛋白,38.9,g/L,L,40.0-55.0,ALB,1
2669,800001979565||B005||1,7412218,4069083,2018-07-24 11:48:00,双相情感障碍，目前为伴有精神病性症状的躁狂发作,血清,2018-07-24 07:05:36,白蛋白,42.4,g/L,N,40.0-55.0,ALB,0
2798,800002012932||B005||1,7459164,4103620,2018-07-27 11:22:00,双相情感障碍，目前为混合性发作,血清,2018-07-27 07:40:37,白蛋白,47.6,g/L,N,40.0-55.0,ALB,0
2905,800002023150||B005||1,7461862,4112018,2018-07-28 11:40:00,双相情感障碍目前为不伴精神病的重度抑郁发作,血清,2018-07-28 07:38:51,白蛋白,45.6,g/L,N,40.0-55.0,ALB,0


#### 球蛋白lgG

In [199]:
df_test_lgG=df_test[df_test['project_name']=='球蛋白']
# 患者限定
df_test_lgG=df_test_lgG[df_test_lgG.case_no.isin(df_VPA_inp_time.case_no.unique())]
# 指标0-1转换
df_test_lgG['肝功_label']=df_test_lgG['test_result'].apply(lambda x: 0 if 20<=float(x)<=40 else 1)

In [200]:
df_test_lgG.head()

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym,肝功_label
2350,800001934594||B005||1,365567,4028318,2018-07-18 13:31:00,双相情感障碍，目前为混合性发作,血清,2018-07-18 07:25:07,球蛋白,24.9,g/L,N,20.0-40.0,G,0
2537,800001953715||B005||1,1819998,4045163,2018-07-20 11:02:00,双相情感障碍，目前为混合性发作,血清,2018-07-20 07:40:33,球蛋白,23.2,g/L,N,20.0-40.0,G,0
2670,800001979565||B005||1,7412218,4069083,2018-07-24 11:48:00,双相情感障碍，目前为伴有精神病性症状的躁狂发作,血清,2018-07-24 07:05:36,球蛋白,33.8,g/L,N,20.0-40.0,G,0
2799,800002012932||B005||1,7459164,4103620,2018-07-27 11:22:00,双相情感障碍，目前为混合性发作,血清,2018-07-27 07:40:37,球蛋白,31.3,g/L,N,20.0-40.0,G,0
2906,800002023150||B005||1,7461862,4112018,2018-07-28 11:40:00,双相情感障碍目前为不伴精神病的重度抑郁发作,血清,2018-07-28 07:38:51,球蛋白,28.2,g/L,N,20.0-40.0,G,0


#### 白球比值AG

In [201]:
df_test_AG=df_test[df_test['project_name'].str.contains('白/球比值')]
# 患者限定
df_test_AG=df_test_AG[df_test_AG.case_no.isin(df_VPA_inp_time.case_no.unique())]
# 指标0-1转换
df_test_AG['肝功_label']=df_test_AG['test_result'].apply(lambda x: 0 if 1.2<=float(x)<=2.4 else 1)

In [202]:
df_test_AG.head()

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym,肝功_label
2351,800001934594||B005||1,365567,4028318,2018-07-18 13:31:00,双相情感障碍，目前为混合性发作,血清,2018-07-18 07:25:07,白/球比值,1.7,None,N,1.2-2.4,A/G,0
2538,800001953715||B005||1,1819998,4045163,2018-07-20 11:02:00,双相情感障碍，目前为混合性发作,血清,2018-07-20 07:40:33,白/球比值,1.7,None,N,1.2-2.4,A/G,0
2671,800001979565||B005||1,7412218,4069083,2018-07-24 11:48:00,双相情感障碍，目前为伴有精神病性症状的躁狂发作,血清,2018-07-24 07:05:36,白/球比值,1.3,None,N,1.2-2.4,A/G,0
2800,800002012932||B005||1,7459164,4103620,2018-07-27 11:22:00,双相情感障碍，目前为混合性发作,血清,2018-07-27 07:40:37,白/球比值,1.5,None,N,1.2-2.4,A/G,0
2907,800002023150||B005||1,7461862,4112018,2018-07-28 11:40:00,双相情感障碍目前为不伴精神病的重度抑郁发作,血清,2018-07-28 07:38:51,白/球比值,1.6,None,N,1.2-2.4,A/G,0


#### 合并肝功能指标

In [293]:
df_test_gangong=pd.concat([df_test_ALT,df_test_AST],axis=0)
df_test_gangong=pd.concat([df_test_gangong,df_test_zhuananmei],axis=0)
df_test_gangong=pd.concat([df_test_gangong,df_test_TP],axis=0)
df_test_gangong=pd.concat([df_test_gangong,df_test_ALB],axis=0)
df_test_gangong=pd.concat([df_test_gangong,df_test_lgG],axis=0)
df_test_gangong=pd.concat([df_test_gangong,df_test_AG],axis=0)

In [294]:
df_test_gangong.project_name.unique()

array(['丙氨酸氨基转移酶', '天门冬氨酸氨基转移酶', '转氨酶比值', '总蛋白', '白蛋白', '球蛋白', '白/球比值'],
      dtype=object)

In [295]:
df_test_gangong.to_excel(project_path+'/data/processed_data/df_6.1.1_肝功能指标.xlsx')

### 皮疹

In [296]:
df_test_pizhen=df_doctor_order[df_doctor_order.drug_name.str.contains('复方酮氯乳膏|莫米松乳膏')]
# 患者限定
df_test_pizhen=df_test_pizhen[df_test_pizhen.case_no.isin(df_VPA_inp_time.case_no.unique())]

In [297]:
df_test_pizhen.drug_name.unique()

array(['复方酮氯乳膏(自产)[20g*1支]', '莫米松乳膏(通用)[5g*1支]', '莫米松乳膏[5g*1支]'],
      dtype=object)

In [298]:
df_test_pizhen.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc
7131,8970838,13022476,复方酮氯乳膏(自产)[20g*1支],1,20g*1支,0.3g,2/日,2021-06-27 10:46:38,NaT,核实
7940,7216247,13317922,复方酮氯乳膏(自产)[20g*1支],1,20g*1支,0.3g,ONCE,2021-07-19 10:42:41,NaT,核实
7941,7216247,13317922,复方酮氯乳膏(自产)[20g*1支],1,20g*1支,0.3g,Tid,2021-07-19 10:46:46,NaT,核实
9817,9290063,14107745,莫米松乳膏(通用)[5g*1支],1,5g*1支,0.1g,1/日,2021-09-29 16:18:17,NaT,核实
16143,6637533,6085020,莫米松乳膏[5g*1支],1,5g*1支,5g,1/日,2019-04-19 10:12:04,NaT,核实


In [299]:
df_test_pizhen.to_excel(project_path+'/data/processed_data/df_6.1.2_皮疹指标.xlsx')

### 恶心

In [300]:
df_test_exin=df_doctor_order[df_doctor_order.drug_name.str.contains('多潘立酮|甲氧氯普胺')]
# 患者限定
df_test_exin=df_test_exin[df_test_exin.case_no.isin(df_VPA_inp_time.case_no.unique())]

In [301]:
df_test_exin.drug_name.unique()

array(['多潘立酮片(吗丁啉)g*42片]', '甲氧氯普胺注射液(胃复安)l/10mg*1支]',
       '甲氧氯普胺片(开封)g*100片]'], dtype=object)

In [302]:
df_test_exin.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc
551,8242965,10130473,多潘立酮片(吗丁啉)[10mg*42片],1,10mg*42片,10mg,ONCE,2020-09-06 16:16:29,NaT,核实
1587,8485991,10389652,多潘立酮片(吗丁啉)[10mg*42片],1,10mg*42片,10mg,ONCE,2020-10-14 23:11:21,NaT,核实
2055,8530956,10588084,甲氧氯普胺注射液(胃复安)[1ml/10mg*1支],1,1ml/10mg*1支,1ml,ONCE,2020-10-23 02:20:58,NaT,核实
5844,8889350,12327058,甲氧氯普胺片(开封)[5mg*100片],1,5mg*100片,5mg,ONCE,2021-04-18 11:04:44,NaT,核实
5938,8889350,12327058,甲氧氯普胺片(开封)[5mg*100片],1,5mg*100片,5mg,ONCE,2021-04-16 12:05:59,NaT,核实


In [303]:
df_test_exin.to_excel(project_path+'/data/processed_data/df_6.1.3_恶心指标.xlsx')

### 血小板计数<100

In [304]:
df_test_PLT=df_test[df_test['project_name'].str.contains('血小板计数')]
# 患者限定
df_test_PLT=df_test_PLT[df_test_PLT.case_no.isin(df_VPA_inp_time.case_no.unique())]
# 指标0-1转换
df_test_PLT['PLT_label']=df_test_PLT['test_result'].apply(lambda x: 1 if float(x)<=100 else 0)

In [305]:
df_test_PLT.project_name.unique()

array(['血小板计数'], dtype=object)

In [306]:
df_test_PLT.to_excel(project_path+'/data/processed_data/df_6.1.2_血小板计数指标.xlsx')

## 安全性指标转置

In [307]:
df_VPA_indicators=df_VPA_inp_time.copy()
df_VPA_indicators=df_VPA_indicators.sort_values(['patient_id','case_no','分裂日期'])
df_VPA_indicators=df_VPA_indicators.reset_index(drop=True)
# 将一次用药的不同检测装置到一行中，比如WBC_before,WBC_3d,WBC_7d
temp_list=[]
for i in df_VPA_indicators.case_no.unique():
    print(i)
    # drug_time
    temp=df_VPA_indicators[df_VPA_indicators['case_no']==i]
    temp=temp.sort_values(['分裂日期']).reset_index(drop=True)
    drug_start_datetime=temp.loc[0,'start_datetime']
    # 肝功数据
    temp_gan=df_test_gangong[df_test_gangong['case_no']==i]
    gan_before=temp_gan[temp_gan['test_date']<=drug_start_datetime]
    gan_after=temp_gan[temp_gan['test_date']>drug_start_datetime]
    if np.sum(gan_before['肝功_label']) >0:
        temp['肝功_before']=1
    else:
        temp['肝功_before']=0
        
    if np.sum(gan_after['肝功_label']) >0:
        temp['肝功_after']=1
    else:
        temp['肝功_after']=0
    # 皮疹数据
    temp_pizhen=df_test_pizhen[df_test_pizhen['case_no']==i]
    pizhen_before=temp_pizhen[temp_pizhen['start_datetime']<=drug_start_datetime]
    pizhen_after=temp_pizhen[temp_pizhen['start_datetime']>drug_start_datetime]
    if pizhen_before.shape[0]>0:
        temp['皮疹_before']=1
    else:
        temp['皮疹_before']=0
    if pizhen_after.shape[0]>0:
        temp['皮疹_after']=1
    else:
        temp['皮疹_after']=0
    # 恶心数据
    temp_exin=df_test_exin[df_test_exin['case_no']==i]
    exin_before=temp_exin[temp_exin['start_datetime']<=drug_start_datetime]
    exin_after=temp_exin[temp_exin['start_datetime']>drug_start_datetime]
    if exin_before.shape[0]>0:
        temp['恶心_before']=1
    else:
        temp['恶心_before']=0
    if exin_after.shape[0]>0:
        temp['恶心_after']=1
    else:
        temp['恶心_after']=0
    # 血小板计数
    temp_PLT=df_test_PLT[df_test_PLT['case_no']==i]
    PLT_before=temp_PLT[temp_PLT['test_date']<=drug_start_datetime]
    PLT_after=temp_PLT[temp_PLT['test_date']>drug_start_datetime]
    if np.sum(PLT_before['PLT_label']) >0:
        temp['PLT_before']=1
    else:
        temp['PLT_before']=0
        
    if np.sum(PLT_after['PLT_label']) >0:
        temp['PLT_after']=1
    else:
        temp['PLT_after']=0
    # 计算安全性指标
    for j in range(temp.shape[0]):
        if np.sum(temp.loc[j,['肝功_before','皮疹_before','恶心_before','PLT_before']])>0:
            temp.loc[j,'security_before']=1
        else:
            temp.loc[j,'security_before']=0

        if np.sum(temp.loc[j,['肝功_after','皮疹_after','恶心_after','PLT_after']])>0:
            temp.loc[j,'security_after']=1        
        else:
            temp.loc[j,'security_after']=0
        # 
        if (temp.loc[j,'security_before']==0) & (temp.loc[j,'security_after']==1):
            temp.loc[j,'security_label']=1
        else:
            temp.loc[j,'security_label']=0
    temp_list.append(temp)

12210690
13397293
4922567
5386601
8981054
7251581
4045163
7661760
9123719
5407981
9818116
11787937
11022153
6435824
7967752
4028318
10028911
7564885
6610460
8150786
7531700
6000741
6338386
5682081
10005574
9574993
4860902
5082652
11747590
8898454
8806185
12855725
5638515
6591824
14322457
4366097
5708206
6667714
5125578
6268920
7079833
14135942
10692337
5375034
5968455
5634908
6486860
8838428
7480196
4151975
6085020
11351405
4267891
5544859
7901590
11291598
10357709
4255479
9910592
4821017
10001302
5302878
4283749
10019847
12313528
7830248
8542045
5612768
7882556
10146112
12631919
5044456
8915869
4553217
13317922
5679938
14040566
9781575
13176502
4918748
6884107
6324940
5653155
7719716
9687004
7728906
10989230
9835190
11547061
4069083
5722142
4103620
4112018
4216082
4133350
13951019
4191397
4236927
10674663
4304073
4579522
4364618
4560293
4446680
4446056
4592169
4536465
4569640
7754134
4707313
8390447
4761372
4842956
4891032
11223919
4926506
5180331
10368285
5278319
13817948
5436932
569

In [308]:
df_VPA_indicators=pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

In [309]:
df_VPA_indicators.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,...,肝功_after,皮疹_before,皮疹_after,恶心_before,恶心_after,PLT_before,PLT_after,security_before,security_after,security_label
0,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2021-03-29 19:14:29,2021-03-29 19:14:29,核实,...,1,0,0,0,0,0,0,0.0,1.0,1.0
1,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-03-31 09:09:50,2021-03-31 09:09:50,核实,...,1,0,0,0,0,0,0,0.0,1.0,1.0
2,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-07 09:10:18,2021-04-07 09:10:18,核实,...,1,0,0,0,0,0,0,0.0,1.0,1.0
3,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-09 19:49:24,2021-04-09 19:49:24,核实,...,1,0,0,0,0,0,0,0.0,1.0,1.0
4,1173560,13397293,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-07-21 16:21:42,2021-07-21 16:21:42,核实,...,1,0,0,0,0,0,0,0.0,1.0,1.0


In [310]:
print(df_VPA_indicators.shape)
print(df_VPA_indicators['patient_id'].nunique())

(3175, 37)
213


In [311]:
df_VPA_indicators.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,...,肝功_after,皮疹_before,皮疹_after,恶心_before,恶心_after,PLT_before,PLT_after,security_before,security_after,security_label
0,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2021-03-29 19:14:29,2021-03-29 19:14:29,核实,...,1,0,0,0,0,0,0,0.0,1.0,1.0
1,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-03-31 09:09:50,2021-03-31 09:09:50,核实,...,1,0,0,0,0,0,0,0.0,1.0,1.0
2,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-07 09:10:18,2021-04-07 09:10:18,核实,...,1,0,0,0,0,0,0,0.0,1.0,1.0
3,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.75,1,2021-04-09 19:49:24,2021-04-09 19:49:24,核实,...,1,0,0,0,0,0,0,0.0,1.0,1.0
4,1173560,13397293,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2021-07-21 16:21:42,2021-07-21 16:21:42,核实,...,1,0,0,0,0,0,0,0.0,1.0,1.0


In [312]:
df_VPA_indicators.to_excel(project_path+'/data/processed_data/df_6.2_安全性指标转置.xlsx')

# 添加丙戊酸TDM检测

## 提取tdm检测数据

In [313]:
# 提取用药后不良反应发生前的甲氨蝶呤血药浓度检测
df_test_VPA=df_test[df_test['project_name'].str.contains('丙戊酸|VPA')]
# 排序
df_test_VPA=df_test_VPA.sort_values(['patient_id','case_no','test_date'])
df_test_VPA=df_test_VPA.reset_index(drop=True)

In [314]:
df_test_VPA['tdm_date']=df_test_VPA['test_date'].apply(lambda x: str(x).split(' ')[0])

In [315]:
df_test_VPA['test_result']=df_test_VPA['test_result'].astype('float')

In [316]:
df_test_VPA.isnull().any()

test_record_id        False
patient_id            False
case_no                True
test_date             False
clinical_diagnosis     True
sample_type           False
sample_time            True
project_name          False
test_result           False
result_unit           False
is_normal             False
refer_scope            True
synonym               False
tdm_date              False
dtype: bool

In [317]:
df_test_VPA=df_test_VPA[df_test_VPA.case_no.notnull()].reset_index(drop=True)

In [318]:
print(df_test_VPA.shape)
print(df_test_VPA['patient_id'].nunique())

(349, 14)
218


In [319]:
df_VPA_indicators.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3175 entries, 0 to 3174
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   patient_id       3175 non-null   object        
 1   case_no          3175 non-null   object        
 2   drug_name        3175 non-null   object        
 3   amount           3175 non-null   object        
 4   drug_spec        3175 non-null   object        
 5   dosage           3175 non-null   object        
 6   frequency        3175 non-null   object        
 7   start_datetime   3175 non-null   datetime64[ns]
 8   end_datetime     3175 non-null   datetime64[ns]
 9   statusdesc       3175 non-null   object        
 10  日剂量              3175 non-null   float64       
 11  start_date       3175 non-null   object        
 12  end_date         3175 non-null   object        
 13  long_d_order     3175 non-null   object        
 14  分裂日期             3175 non-null   object 

In [320]:
# 提取患者的tdm检测
df_test_VPA_choose=df_test_VPA[df_test_VPA['patient_id'].isin(df_VPA_indicators.patient_id.unique())]
df_test_VPA_choose=df_test_VPA_choose.sort_values(['patient_id','test_date'])
df_test_VPA_choose=df_test_VPA_choose.reset_index(drop=True)

In [321]:
print(df_test_VPA_choose.shape)
print(df_test_VPA_choose['patient_id'].nunique())
print(df_test_VPA_choose['case_no'].nunique())

(342, 14)
213
236


In [322]:
# 保存丙戊酸血药浓度检测
df_test_VPA_choose.to_excel(project_path+'/data/processed_data/df_7.1_test_VPA.xlsx')

## 合并tdm检测数据

In [323]:
import datetime
# 合并甲氨蝶呤tdm检测信息
# 所有tdm检测都认为是稳态检测。
temp_list=[]
for i in np.unique(df_VPA_indicators['case_no']):
    # 稳态 丙戊酸 temp
    temp_steady=df_VPA_indicators[df_VPA_indicators['case_no']==i]
    temp_steady=temp_steady.sort_values(['patient_id','case_no','分裂日期'])
    temp_steady=temp_steady.reset_index(drop=True)
    
    # tdm temp
    temp_tdm=df_test_VPA_choose[df_test_VPA_choose['case_no']==i]
    temp_tdm=temp_tdm.sort_values(['patient_id','case_no','test_date'])
    temp_tdm=temp_tdm.reset_index(drop=True)
    # tdm都认为是稳态浓度，不考虑3天稳态
    for j in range(temp_tdm.shape[0]):
        # 提取tdm检测日期
        tdm_date=pd.to_datetime(temp_tdm.loc[j,'tdm_date'])
#         print(tdm_date)
#         # 提取稳态前tdm前3天用药
#         temp_before_drug=temp_before[(pd.to_datetime(temp_before['分裂日期'])<=tdm_date-datetime.timedelta(days=1))&
#                                     (pd.to_datetime(temp_before['分裂日期'])>=tdm_date-datetime.timedelta(days=3))]
        # 提取tdm前3天用药
        temp_steady_drug=temp_steady[(pd.to_datetime(temp_steady['分裂日期'])<=tdm_date-datetime.timedelta(days=1)) &
                                    (pd.to_datetime(temp_steady['分裂日期'])>=tdm_date-datetime.timedelta(days=4))]
        # 如果稳态前和稳态时用药有变化，则舍弃tdm数据，否则，并入到VPA用药数据中
#         if temp_before_drug.shape[0] != temp_steady_drug.shape[0]:
#             continue
#         else:
        if temp_steady_drug.shape[0]>0:
            # 获取最近用药的索引
            index=temp_steady_drug.index.tolist()[-1]
            temp_steady.loc[index,'tdm_date']=temp_tdm.loc[j,'tdm_date']
            temp_steady.loc[index,'test_result']=temp_tdm.loc[j,'test_result']
    temp_list.append(temp_steady)

In [324]:
tdm_date

Timestamp('2020-08-14 00:00:00')

In [325]:
df_VPA_tdm = pd.concat(temp_list,axis=0).reset_index(drop=True)
# df_VPA_tdm = df_VPA_tdm.drop_duplicates().reset_index(drop=True)
del temp_list

In [326]:
print(df_VPA_tdm.shape)
print(df_VPA_tdm['patient_id'].nunique())
print(df_VPA_tdm['case_no'].nunique())

(3175, 39)
213
236


In [327]:
df_VPA_tdm.columns

Index(['patient_id', 'case_no', 'drug_name', 'amount', 'drug_spec', 'dosage',
       'frequency', 'start_datetime', 'end_datetime', 'statusdesc', '日剂量',
       'start_date', 'end_date', 'long_d_order', '分裂日期', '起始标签', '时间标签',
       'gender', 'age', '身高', '体重', 'BMI', '基础疾病', 'adm_date', 'dis_date',
       '住院时长', '肝功_before', '肝功_after', '皮疹_before', '皮疹_after', '恶心_before',
       '恶心_after', 'PLT_before', 'PLT_after', 'security_before',
       'security_after', 'security_label', 'tdm_date', 'test_result'],
      dtype='object')

In [328]:
# 保存添加tdm检测的甲氨蝶呤用药
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_7.2_添加tdm检测.xlsx')
df_VPA_tdm.to_excel(writer)
writer.save()

## 删除tdm为空的数据

In [329]:
df_VPA_tdm=df_VPA_tdm[df_VPA_tdm['test_result'].notnull()].reset_index(drop=True)

In [330]:
print(df_VPA_tdm.shape)
print(df_VPA_tdm['patient_id'].nunique())
print(df_VPA_tdm['case_no'].nunique())

(270, 39)
177
196


In [331]:
# 保存tdm非空的用药
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_7.3_tdm非空用药.xlsx')
df_VPA_tdm.to_excel(writer)
writer.save()

## 插入有效性标签

In [332]:
df_VPA_tdm['efficiency_label']=df_VPA_tdm['test_result'].apply(lambda x: 1 if 50<=float(x)<=100 else 0)

In [333]:
df_VPA_tdm.to_excel(project_path+'/data/processed_data/df_7.4_插入有效性标签.xlsx')

## 提取安全有效的用药数据

In [334]:
# 安全标签为1，有效标签为1
df_VPA_tdm_choose=df_VPA_tdm[(df_VPA_tdm['security_label']==1) & (df_VPA_tdm['efficiency_label']==1)]

In [335]:
print(df_VPA_tdm_choose.shape)
print(df_VPA_tdm_choose['patient_id'].nunique())

(151, 40)
110


In [336]:
df_VPA_tdm_choose.to_excel(project_path+'/data/processed_data/df_7.4_筛选安全有效的用药数据.xlsx')

In [337]:
# 非安全有效指标
df_VPA_tdm_no=df_VPA_tdm[~ ((df_VPA_tdm['security_label']==1) & (df_VPA_tdm['efficiency_label']==1))]

In [338]:
print(df_VPA_tdm_no.shape)
print(df_VPA_tdm_no['patient_id'].nunique())

(119, 40)
95


In [339]:
# ## 统计用药和tdm先后

# # 丙戊酸首次用药
# # 排序
# df_VPA=df_VPA.sort_values(['patient_id','case_no','start_datetime'])
# # 取第一次用药
# df_VPA_first=df_VPA.drop_duplicates(['case_no'],keep='first')

# df_VPA_first.info()

# print(df_VPA_first.shape)
# print(df_VPA_first['patient_id'].nunique())

# # tdm首次检测
# # 排序
# df_test_VPA=df_test_VPA.sort_values(['patient_id','case_no','test_date'])
# # 取第一次tdm
# df_test_VPA_first=df_test_VPA.drop_duplicates(['case_no'],keep='first')
# df_test_VPA_first['case_no']=df_test_VPA_first['case_no'].astype(str)

# df_test_VPA_first.info()

# print(df_test_VPA_first.shape)
# print(df_test_VPA_first['patient_id'].nunique())

# # tdm时间早于用药的id
# temp_list=[]
# for i in df_VPA_first.case_no.unique():
#     temp_VPA=df_VPA_first[df_VPA_first['case_no']==i]
#     temp_VPA=temp_VPA.reset_index(drop=True)
#     temp_tdm=df_test_VPA_first[df_test_VPA_first['case_no']==i]
#     break
#     temp_tdm=temp_tdm.reset_index(drop=True)
#     if temp_VPA.shape[0]>0:
#         if temp_VPA.loc[0,'start_datetime'] >= temp_tdm.loc[0,'test_date']:
#             temp_list.append(temp_VPA)

# df_temp_compare = pd.concat(temp_list,axis=0).reset_index(drop=True)
# del temp_list

# print(df_temp_compare.shape)
# print(df_temp_compare['patient_id'].nunique())

# df_temp_compare.to_excel(project_path+'/data/processed_data/df_7.4_比较tdm和用药时间先后.xlsx')

## 统计tdm检测前用药天数

In [340]:
# 纳排后不同用药稳态在tdm检测前天数

In [341]:
# 1天用药稳态
df_VPA_steady_1d=df_VPA_modification.copy()

In [342]:
# 2天用药稳态
df_VPA_steady_2d=df_VPA_modification.copy()
# 到稳态血药浓度（同一剂量至少连续服用3天)
temp_list=[]
for i in df_VPA_steady_2d['case_no'].unique():
    print(i)
    temp=df_VPA_steady_2d[df_VPA_steady_2d['case_no']==i]
    temp=temp.reset_index(drop=True)
    j=0
    while j < temp.shape[0]:
#         print(j)
        # 连续服药3天，tdm达到稳态。1.剂量连续3天；2. 时间连续3天
        dosage_judge=temp.loc[j,'日剂量']    
        # 1. 剂量判断
        break_bool=False
        for k in range(j+1,temp.shape[0]):
            differ = (pd.to_datetime(temp.loc[k,'分裂日期'])-pd.to_datetime(temp.loc[k-1,'分裂日期'])).days
            if dosage_judge != temp.loc[k,'日剂量'] or differ != 1:
                # 判断是否中断
                break_bool=True
                break
        # 是否是最后一天
        if k != temp.shape[0]-1:
            # 是否是稳态
            if k-1-j>=1:
                temp_list.append(temp.loc[j:k-1,])
            j=k
            continue
        else:
            # 最后一天是否中断
            if break_bool:
                # 是否是稳态
                if k-1-j>=1:
                    temp_list.append(temp.loc[j:k-1,])
            else:
                if k-j >=1:
                    temp_list.append(temp.loc[j:k,])
            j=k+1
# list转换为DataFrame
df_VPA_steady_2d = pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

10001302
10005574
10019847
10024628
10028911
10130473
10146112
10171606
10200584
10293806
10297961
10338392
10343367
10357709
10368285
10389652
10526969
10559425
10582637
10588084
10616235
10674663
10692337
10830666
10859517
10864711
10877290
10914758
10941636
10989230
11022153
11069250
11103754
11200975
11223919
11238946
11291598
11351405
11547061
11747590
11787937
12001085
12055514
12069121
12070485
12165203
12210690
12238352
12297734
12313528
12325442
12327058
12415154
12550325
12610841
12631919
12693940
12696147
12782405
12835527
12855725
13022476
13063464
13150462
13176502
13266036
13317922
13335769
13397293
13425571
13491055
13561501
13692578
13817948
13951019
13997754
14040566
14103871
14107745
14120909
14135942
14184132
14322457
14400275
14701280
14712892
14900947
15026052
4028318
4045163
4069083
4103620
4112018
4133350
4151975
4191397
4216082
4236927
4255479
4267891
4283749
4304073
4364618
4366097
4446056
4446680
4536465
4553217
4560293
4569640
4579522
4592169
4707313
4761372


In [343]:
print(df_VPA_steady_2d.shape)
print(df_VPA_steady_2d['patient_id'].nunique())

(2522, 17)
155


In [344]:
df_VPA_steady_2d.to_excel(project_path+'/data/df_VPA_steady_2d.xlsx')

In [345]:
# 同一剂量3天连续用药
df_VPA_steady=df_VPA_modification.copy()
# 到稳态血药浓度（同一剂量至少连续服用3天)
temp_list=[]
for i in df_VPA_steady['case_no'].unique():
    print(i)
    temp=df_VPA_steady[df_VPA_steady['case_no']==i]
    temp=temp.reset_index(drop=True)
    j=0
    while j < temp.shape[0]:
#         print(j)
        # 连续服药3天，tdm达到稳态。1.剂量连续3天；2. 时间连续3天
        dosage_judge=temp.loc[j,'日剂量']    
        # 1. 剂量判断
        break_bool=False
        for k in range(j+1,temp.shape[0]):
            differ = (pd.to_datetime(temp.loc[k,'分裂日期'])-pd.to_datetime(temp.loc[k-1,'分裂日期'])).days
            if dosage_judge != temp.loc[k,'日剂量'] or differ != 1:
                # 判断是否中断
                break_bool=True
                break
        # 是否是最后一天
        if k != temp.shape[0]-1:
            # 是否是稳态
            if k-1-j>=2:
                temp_list.append(temp.loc[j:k-1,])
            j=k
            continue
        else:
            # 最后一天是否中断
            if break_bool:
                # 是否是稳态
                if k-1-j>=2:
                    temp_list.append(temp.loc[j:k-1,])
            else:
                if k-j >=2:
                    temp_list.append(temp.loc[j:k,])
            j=k+1

# list转换为DataFrame
df_VPA_steady = pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

10001302
10005574
10019847
10024628
10028911
10130473
10146112
10171606
10200584
10293806
10297961
10338392
10343367
10357709
10368285
10389652
10526969
10559425
10582637
10588084
10616235
10674663
10692337
10830666
10859517
10864711
10877290
10914758
10941636
10989230
11022153
11069250
11103754
11200975
11223919
11238946
11291598
11351405
11547061
11747590
11787937
12001085
12055514
12069121
12070485
12165203
12210690
12238352
12297734
12313528
12325442
12327058
12415154
12550325
12610841
12631919
12693940
12696147
12782405
12835527
12855725
13022476
13063464
13150462
13176502
13266036
13317922
13335769
13397293
13425571
13491055
13561501
13692578
13817948
13951019
13997754
14040566
14103871
14107745
14120909
14135942
14184132
14322457
14400275
14701280
14712892
14900947
15026052
4028318
4045163
4069083
4103620
4112018
4133350
4151975
4191397
4216082
4236927
4255479
4267891
4283749
4304073
4364618
4366097
4446056
4446680
4536465
4553217
4560293
4569640
4579522
4592169
4707313
4761372


In [346]:
df_aaa=pd.DataFrame(data=[],columns=['aaa','bbb'])
print(df_aaa.shape[0])

0


In [347]:
# 3天用药稳态
drug_days_list=[]
case_no_list=[]
for i in df_VPA_steady_1d.case_no.unique():
#     print(i)
    case_no_list.append(i)
    # 1天稳态
    temp_VPA_1d=df_VPA_steady_1d[df_VPA_steady_1d['case_no']==i]
    temp_VPA_1d=temp_VPA_1d.reset_index(drop=True)
    # 2天稳态
    temp_VPA_2d=df_VPA_steady_2d[df_VPA_steady_2d['case_no']==i]
    temp_VPA_2d=temp_VPA_2d.reset_index(drop=True)
    # 3天稳态
    temp_VPA=df_VPA_steady[df_VPA_steady['case_no']==i]
    temp_VPA=temp_VPA.reset_index(drop=True)
    # tdm检测
    temp_tdm=df_test_VPA[df_test_VPA['case_no']==i]
    temp_tdm=temp_tdm.reset_index(drop=True)
    day_list=[]
    for j in range(temp_tdm.shape[0]):
        # 1天稳态
        temp2=temp_VPA_1d[(pd.to_datetime(temp_VPA_1d['分裂日期'])>=pd.to_datetime(temp_tdm.loc[j,'tdm_date'])-datetime.timedelta(days=3))&
                          (temp_VPA_1d['分裂日期']<temp_tdm.loc[j,'tdm_date'])]
        if temp2.shape[0]<1:
            day_list.append(0)
        else:
            day_list.append(1)
        # 2天稳态
        temp2=temp_VPA_2d[(pd.to_datetime(temp_VPA_2d['分裂日期'])>=pd.to_datetime(temp_tdm.loc[j,'tdm_date'])-datetime.timedelta(days=2))&
                          (temp_VPA_2d['分裂日期']<temp_tdm.loc[j,'tdm_date'])]
        if temp2.shape[0]<2:
            day_list.append(0)
        else:
            day_list.append(1)
        # 3天稳态
        temp2=temp_VPA[(pd.to_datetime(temp_VPA['分裂日期'])>=pd.to_datetime(temp_tdm.loc[j,'tdm_date'])-datetime.timedelta(days=3))&
                      (temp_VPA['分裂日期']<temp_tdm.loc[j,'tdm_date'])]
        if temp2.shape[0]<3:
            day_list.append(0)
        else:
            day_list.append(1)
    drug_days_list.append(day_list)

In [348]:
df_tdm_days_count=pd.DataFrame(data=drug_days_list,index=case_no_list,columns=['1d_tdm1','2d_tdm1','3d_tdm1',
                                                                               '1d_tdm2','2d_tdm2','3d_tdm2',
                                                                               '1d_tdm3','2d_tdm3','3d_tdm3',
                                                                               '1d_tdm4','2d_tdm4','3d_tdm4',
                                                                               '1d_tdm5','2d_tdm5','3d_tdm5'])

In [349]:
df_tdm_days_count.to_excel(project_path+'/data/processed_data/df_7.5_统计tdm检测前用药天数.xlsx')

# 增加联合用药

## 提取联合用药

In [350]:
# 提取患者的联合用药
df_drug_other=df_doctor_order[~ df_doctor_order['drug_name'].str.contains('丙戊酸|葡萄糖|氯化钠')]
# 患者限定
df_drug_other=df_drug_other[df_drug_other['case_no'].isin(df_VPA_tdm['case_no'].unique())]
df_drug_other=df_drug_other.reset_index(drop=True)

In [351]:
print(df_drug_other.shape)
print(df_drug_other['patient_id'].nunique())

(19916, 10)
177


In [352]:
df_drug_other

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc
0,6873870,10001302,齐拉西酮冻干粉针(江都医院)[30mg*1瓶],1,30mg*1瓶,20mg,ONCE,2020-08-15 18:59:57,NaT,核实
1,6873870,10001302,行为矫正治疗,20,NaN,NaN,1/日,2020-08-15 19:45:12,2020-09-04 11:38:00,停止
2,6873870,10001302,静脉输液第二组起每组,3,NaN,NaN,1/日,2020-08-17 11:24:51,2020-08-20 10:11:04,停止
3,6873870,10001302,脑苷肌肽注射液(欧迪美)[2ml*1支],6,2ml*1支,4ml,1/日,2020-08-17 11:25:27,2020-08-20 10:11:09,停止
4,6873870,10001302,静脉输液第二组起每组,3,NaN,NaN,1/日,2020-08-17 11:25:27,2020-08-20 10:11:09,停止
...,...,...,...,...,...,...,...,...,...,...
19911,7692024,14900947,2*一次性使用无菌输液接头JT-SJ-WZ,1,NaN,NaN,ONCE,2021-12-04 09:02:34,NaT,核实
19912,7692024,14900947,静脉留置针护理(含换药、封管),1,NaN,NaN,1/日,2021-12-04 09:03:08,NaT,核实
19913,7692024,14900947,★富马酸喹硫平缓释片(百朗欣)[0.2g*20片],1,0.2g*20片,0.4g,1/日,2021-12-05 08:39:56,NaT,核实
19914,7692024,14900947,★富马酸喹硫平缓释片(百朗欣)[0.2g*20片],1,0.2g*20片,0.4g,ONCE,2021-12-05 10:40:00,NaT,核实


In [353]:
df_drug_other.to_excel(project_path+'/data/processed_data/df_8.1_drug_other.xlsx')

## 统计联合用药频数

In [354]:
# 统计用药频数，由业务人员确定选用哪些联合用药。
# 联合用药时间限定：结束时间>=用药前3天内+开始时间<=tdm检测前，取交集
df_VPA_drug_other=df_VPA_tdm.copy()
temp_list=[]
for i in range(df_VPA_drug_other.shape[0]):
    case_no=df_VPA_drug_other.loc[i,'case_no']
    # 联合用药
    drug_other=df_drug_other[df_drug_other['case_no']==case_no]
    drug_other=drug_other.reset_index(drop=True)
    # 分裂联合用药日期
    drug_other['start_date']=drug_other['start_datetime'].apply(lambda x: pd.to_datetime(str(x).split(' ')[0])) 
    drug_other['end_date']=drug_other['end_datetime'].apply(lambda x: pd.to_datetime(str(x).split(' ')[0])) 
    # 时间限定：用药前-tdm检测前
    drug_time=df_VPA_drug_other.loc[i,'分裂日期']
    tdm_time=df_VPA_drug_other.loc[i,'tdm_date']
    # 提取用药前3天内-tdm前的联合用药。
    drug_other_1=drug_other[(drug_other['end_date']>=pd.to_datetime(tdm_time)-datetime.timedelta(days=4)) &
                         (drug_other['start_date']<=pd.to_datetime(tdm_time))]
    temp_list.append(drug_other_1)

df_drug_other_choose = pd.concat(temp_list,axis=0)
del temp_list
df_drug_other_choose=df_drug_other_choose.drop_duplicates().reset_index(drop=True)
drug_name_counts=df_drug_other_choose.drug_name.value_counts()
df_drug_name_counts=pd.DataFrame(data={'药物名称':drug_name_counts.index,
                                     '频次':drug_name_counts.values})
print(df_drug_name_counts.shape)
# 保存联合用药名
df_drug_name_counts.to_excel(project_path+'/data/processed_data/df_8.2_统计联合用药频数.xlsx')

(164, 2)


## 合并联合用药

In [355]:
# 业务人员给定的联合用药
emotion_wending=['拉莫三嗪', '氯硝西泮', '地西泮', '氢溴酸伏硫西汀片', '阿戈美拉汀', '西酞普兰']
anti_psychotic=['奥氮平', '喹硫平', '阿立哌唑', '齐拉西酮', '利培酮', '帕利哌酮', '氯氮平', '氯丙嗪']
anti_anxiety=['劳拉西泮', '阿普唑仑', '坦度螺酮', '舍曲林片', '艾司西酞普兰', '氟西汀', '帕罗西汀', '米氮平', '曲唑酮']
sleeping_pills=['右佐匹克隆', '唑吡坦', '舒眠胶囊', '乌灵胶囊', '艾司唑仑']
behavior_therapy=['暗示治疗', '引导式教育训练', '行为矫正治疗', '冲动行为干预治疗']

In [356]:
df_emotion_wending=df_drug_other_choose[df_drug_other_choose.drug_name.str.contains('拉莫三嗪|氯硝西泮|地西泮|氢溴酸伏硫西汀片|阿戈美拉汀|西酞普兰')]
df_emotion_wending.to_excel(project_path+'/data/processed_data/df_emotion_wending.xlsx')

In [357]:
df_anti_psychotic=df_drug_other_choose[df_drug_other_choose.drug_name.str.contains('奥氮平|喹硫平|阿立哌唑|齐拉西酮|利培酮|帕利哌酮|氯氮平|氯丙嗪')]
df_anti_psychotic.to_excel(project_path+'/data/processed_data/df_anti_psychotic.xlsx')

In [358]:
df_anti_anxiety=df_drug_other_choose[df_drug_other.drug_name.str.contains('劳拉西泮|阿普唑仑|坦度螺酮|舍曲林片|艾司西酞普兰|氟西汀|帕罗西汀|米氮平|曲唑酮')]
df_anti_anxiety.to_excel(project_path+'/data/processed_data/df_anti_anxiety.xlsx')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [359]:
df_sleeping_pills=df_drug_other_choose[df_drug_other_choose.drug_name.str.contains('右佐匹克隆|唑吡坦|舒眠胶囊|乌灵胶囊|艾司唑仑')]
df_sleeping_pills.to_excel(project_path+'/data/processed_data/df_sleeping_pills.xlsx')

In [360]:
df_behavior_therapy=df_drug_other_choose[df_drug_other_choose.drug_name.str.contains('暗示治疗|引导式教育训练|行为矫正治疗|冲动行为干预治疗')]
df_behavior_therapy.to_excel(project_path+'/data/processed_data/df_behavior_therapy.xlsx')

In [361]:
df_VPA_drug_other.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 40 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   patient_id        270 non-null    object        
 1   case_no           270 non-null    object        
 2   drug_name         270 non-null    object        
 3   amount            270 non-null    object        
 4   drug_spec         270 non-null    object        
 5   dosage            270 non-null    object        
 6   frequency         270 non-null    object        
 7   start_datetime    270 non-null    datetime64[ns]
 8   end_datetime      270 non-null    datetime64[ns]
 9   statusdesc        270 non-null    object        
 10  日剂量               270 non-null    float64       
 11  start_date        270 non-null    object        
 12  end_date          270 non-null    object        
 13  long_d_order      270 non-null    object        
 14  分裂日期              270 non-

In [362]:
# 联合用药时间限定：结束时间>=用药前3天内+开始时间<=tdm检测前，取交集
df_VPA_drug_other=df_VPA_tdm.copy()
df_VPA_drug_other=df_VPA_drug_other.reset_index(drop=True)
for i in range(df_VPA_drug_other.shape[0]):
    case_no=df_VPA_drug_other.loc[i,'case_no']
    # 联合用药
    drug_other=df_drug_other[df_drug_other['case_no']==case_no]
    drug_other=drug_other.reset_index(drop=True)
    # 分裂联合用药日期
    drug_other['start_date']=drug_other['start_datetime'].apply(lambda x: pd.to_datetime(str(x).split(' ')[0])) 
    drug_other['end_date']=drug_other['end_datetime'].apply(lambda x: pd.to_datetime(str(x).split(' ')[0])) 
    # 结束时间>=用药前3天内+开始时间<=tdm检测前，取交集
    drug_time=df_VPA_drug_other.loc[i,'分裂日期']
    tdm_time=df_VPA_drug_other.loc[i,'tdm_date']
    drug_other_1=drug_other[(drug_other['end_date']>=pd.to_datetime(tdm_time)-datetime.timedelta(days=4)) &
                         (drug_other['start_date']<=pd.to_datetime(tdm_time))]
    # 联合用药列表
    drug_other_list=drug_other_1.drug_name.unique()
    # 联合用药转置
    selet_drug_other=['情感稳定剂','抗精神病药','抗焦虑药','安眠药','行为干预治疗']
    drug_other_name_list=[emotion_wending,anti_psychotic,anti_anxiety,sleeping_pills,behavior_therapy]
    for name,name_list in zip(selet_drug_other,drug_other_name_list):
        match_value=[1 if re.match(x,y) else 0 for x in name_list for y in drug_other_list]
        if np.sum(match_value)>0:
            df_VPA_drug_other.loc[i,name]=1
        else:
            df_VPA_drug_other.loc[i,name]=0

In [363]:
print(df_VPA_drug_other.shape)
print(df_VPA_drug_other['patient_id'].nunique())

(270, 45)
177


In [364]:
# 时间限定的联合用药
df_VPA_drug_other.to_excel(project_path+'/data/processed_data/df_8.3_合并联合用药.xlsx')

# 合并其他相关检测

## 提取其他检测

In [365]:
# 提取甲氨蝶呤患者的所有检测
df_test_other=df_test[df_test['case_no'].isin(df_VPA_drug_other['case_no'].unique())]

In [366]:
# 提取患者的其他检测
df_test_other = df_test_other[~ df_test_other['project_name'].str.contains('丙戊酸')]
df_test_other = df_test_other.reset_index(drop=True)

In [383]:
df_test_other.isnull().any()

test_record_id        False
patient_id            False
case_no               False
test_date             False
clinical_diagnosis     True
sample_type           False
sample_time            True
project_name          False
test_result           False
result_unit            True
is_normal              True
refer_scope            True
synonym                True
dtype: bool

In [384]:
print(df_test_other.shape)
print(df_test_other['patient_id'].nunique())

(29494, 13)
177


In [385]:
# 保存其他检测
df_test_other.to_csv(project_path+'/data/processed_data/df_9.1_test_other.csv')

## 统计其他检测项频数

In [386]:
df_test_other_name=pd.DataFrame(df_test_other['project_name'].value_counts())

In [387]:
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_9.2_统计其他检测项频数.xlsx')
df_test_other_name.to_excel(writer)
writer.save()

## 合并其他检测

In [388]:
def judge_float(x):
    try:
        a=float(x)
        return a
    except:
        return np.NaN

In [389]:
df_VPA_test_other=df_VPA_drug_other.copy()
# 将用药后不良反应发生前的最近一次其他检测转置到一行中
for i in range(df_VPA_test_other.shape[0]):
    case_no=df_VPA_test_other.loc[i,'case_no']
    drug_time=df_VPA_test_other.loc[i,'分裂日期']
    tdm_date=df_VPA_test_other.loc[i,'tdm_date']
    # case_no的其他检测
    test_other= df_test_other[df_test_other['case_no']==case_no]
    test_other=test_other.reset_index(drop=True)
    test_other['test_date']=test_other['test_date'].apply(lambda x: pd.to_datetime(str(x).split(' ')[0])) # 分裂检测日期
    # 取用药后-不良反应发生前的其他检测，此案例中start_datetime>tdm_date-1。
    test_other_1=test_other[(test_other['test_date']>=pd.to_datetime(tdm_date)-datetime.timedelta(days=4))&
                            (test_other['test_date']<=tdm_date)]
    # 取最近的一次其他检测
    test_other_1=test_other_1.sort_values(['test_date'])
    test_other_1=test_other_1.drop_duplicates(['project_name'],keep='last')
    test_other_1=test_other_1.reset_index(drop=True)
    
    # 检验项转置为表头
    if test_other_1.shape[0]>0:
        test_other_1['test_result']=test_other_1['test_result'].apply(judge_float)
        test_other_pivot=test_other_1.pivot_table('test_result',['case_no'], 'project_name')
        test_other_pivot=test_other_pivot.reset_index(drop=True)
        for j in test_other_pivot.columns[1:]:
            df_VPA_test_other.loc[i,j]=test_other_pivot.loc[0,j]
#     break
#     if case_no == '410308':
#         print(temp.loc[0,'start_datetime'])
#         print(temp.loc[0,'bmd_time'])
#         print(test_other_1)
#         print(test_other_2[test_other_2['project_name']=='肌酐(干式)'])
#         break

In [390]:
print(df_VPA_test_other.shape)
print(df_VPA_test_other['patient_id'].nunique())

(270, 178)
177


In [375]:
# 合并其他检测
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_9.3_合并其他检测.xlsx')
df_VPA_test_other.to_excel(writer)
writer.save()

## 删除缺失率超过50%的列

In [398]:
df_VPA_test_other.columns

Index(['patient_id', 'case_no', 'drug_name', 'amount', 'drug_spec', 'dosage',
       'frequency', 'start_datetime', 'end_datetime', 'statusdesc',
       ...
       '碱性磷酸酶', '红细胞沉降率测定', '胆碱脂酶', '乙肝病毒表面抗原', '艾滋病病毒抗体', '抗甲状腺球蛋白抗体',
       '抗甲状腺过氧化物酶', '降钙素原', '促甲状腺受体自身抗体', '镜检红细胞'],
      dtype='object', length=172)

In [399]:
# 删除缺失超过50%的检测
for i in df_VPA_test_other.columns:
    x=df_VPA_test_other[i].isnull().sum()
    per_x = x/df_VPA_test_other.shape[0]  
    if per_x >= 0.5:
        print(i)
        del df_VPA_test_other[i]

-IG#
-IG%
-LYZ
-MicroR
-NESFL
-NEWX
中性粒细胞总数
中性粒细胞百分数
单核细胞总数
单核细胞百分数
嗜碱性粒细胞总数
嗜碱性粒细胞百分数
嗜酸性粒细胞总数
嗜酸性粒细胞百分数
大型血小板比值
平均RBC血红蛋白浓度
平均红细胞体积
平均红细胞血红蛋白量
平均血小板容积
淋巴细胞总数
淋巴细胞百分数
白细胞计数
红细胞分布宽度-CV
红细胞分布宽度-SD
红细胞比积测定
红细胞计数
血小板分布宽度
血小板比积测定
血小板计数
血红蛋白测定
天门冬氨酸氨基转移酶
总胆红素
总蛋白
球蛋白
白/球比值
白蛋白
直接胆红素
转氨酶比值
间接胆红素
催乳素
卵泡刺激素
可溶性转铁蛋白受体
孕酮
总铁结合力
睾酮
转铁蛋白
转铁蛋白饱和度
铁离子
铁蛋白
雌二醇
黄体生成素
不饱和铁结合力
丙型肝炎病毒抗体[电化学发光]
丙氨酸氨基转移酶
乙型肝炎e抗体测定
乙型肝炎e抗原测定
乙型肝炎核心抗体测定
乙型肝炎表面抗体测定
乙型肝炎表面抗原测定
低密度脂蛋白胆固醇
促甲状腺素
凝血酶原时间比率
凝血酶原活动度
凝血酶时间测定
国际标准比率
小而密低密度脂蛋白胆固醇
尿比重(拆射计法)
尿素
尿酸
心型肌酸激酶
总胆固醇
总钙
极低密度脂蛋白胆固醇
梅毒螺旋体特异性抗体[电化学发光法]
活化部分凝血活酶时间
游离T3
游离T4
甘油三酯
甲状三碘原氨酸
甲状腺素
白介素-6
肌酐
肌酸激酶
胱抑素-C
脂蛋白a
葡萄糖
血浆D-二聚体测定
血浆凝血酶原时间测定
血浆纤维蛋白原测定
血清载脂蛋白E测定
载脂蛋白A
载脂蛋白B
载脂蛋白比值
酸碱浓度
高密度脂蛋白胆固醇
乳酸脱氢酶
β人绒毛膜促性腺激素
尿沉渣镜检白细胞
a-羟丁酸脱氢酶
糖化血红蛋白
肾小球滤过率(CKD-EPI公式)
非高密度脂蛋白胆固醇
皮质醇
脂肪酶
尿沉渣镜检红细胞
r-谷氨酰基转移酶
前白蛋白
同型半胱氨酸
尿上皮细胞(仪器定量)
尿白细胞(仪器定量)
尿管型(仪器定量)
尿红细胞(仪器定量)
尿细菌(仪器定量)
尿结晶
尿酵母菌
岩藻糖苷酶
总胆汁酸
碱性磷酸酶
红细胞沉降率测定
胆碱脂酶
乙肝病毒表面抗原
艾滋病病毒抗体
抗甲状腺球蛋白抗体
抗甲状腺过氧化物酶
降钙素原
促甲状腺受体自身抗体
镜检红细胞


In [400]:
print(df_VPA_test_other.shape)
print(df_VPA_test_other['patient_id'].nunique())

(270, 45)
177


In [401]:
# 保存删除缺失率后的数据
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_9.4_删除缺失超过50%的列.xlsx')
df_VPA_test_other.to_excel(writer)
writer.save()

In [402]:
df_VPA_test_other.日剂量.value_counts()

1.00    184
0.50     40
1.50     23
2.00      9
0.75      6
1.25      5
0.25      2
2.25      1
Name: 日剂量, dtype: int64

## 人工删除没有意义的指标

In [403]:
del df_VPA_test_other['氯离子']
del df_VPA_test_other['无机磷']
del df_VPA_test_other['总二氧化碳']
del df_VPA_test_other['钠离子']
del df_VPA_test_other['钾离子']
del df_VPA_test_other['镁离子']

KeyError: '氯离子'

In [404]:
df_VPA_test_other.to_excel(project_path+'/data/processed_data/df_9.5_人工删除没有意义的指标.xlsx')